Use Ampere architecture or cpu. Hopper architecture does not support this old environment.

```bash
git clone https://github.com/HayatoHongo/LLaVA.git
cd LLaVA/

wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda.sh
bash ~/miniconda.sh # Enter, yes, Enter, Enter

source ~/.bashrc
source ~/miniconda3/etc/profile.d/conda.sh
conda --version
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r
conda create -n llava python=3.10.12 -y

conda activate llava

pip --version # make sure you use pip 25.1
cd LLaVA/
pip install -e .
pip install -e ".[train]"
pip install flash-attn==2.3.4 --no-build-isolation
pip install ipykernel

# Jupyter 用のカーネルを建てる
python -m ipykernel install --user \
  --name llava \
  --display-name "Jupyter (llava)"

python -c "import deepspeed; print(deepspeed.__version__)" # make sure you are using deepspeed 0.9.5

# そもそも作成したJupyter用のカーネルが認識されていないので、Ctrl+Shift+P (Cmd+Shift+P for Mac)でReload Window
# 一度SSHを切って再接続する。
# 再度仮想環境をactivate
conda activate llava

# カーネルでjupyter用のカーネルを選択する。Python環境ではないことに注意。

In [3]:
!python -c "import torch, transformers; print(torch.__version__, transformers.__version__)"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'transformers'


In [4]:
import transformers
import torch
from dataclasses import dataclass, field
from typing import Optional

In [5]:
@dataclass
class ModelArguments:
    model_name_or_path: Optional[str] = field(default="facebook/opt-125m")
    version: Optional[str] = field(default="v0")
    freeze_backbone: bool = field(default=False)
    tune_mm_mlp_adapter: bool = field(default=False)
    vision_tower: Optional[str] = field(default=None) # default to None
    mm_vision_select_layer: Optional[int] = field(default=-1)   # default to the last layer
    pretrain_mm_mlp_adapter: Optional[str] = field(default=None)
    mm_projector_type: Optional[str] = field(default='linear')
    mm_use_im_start_end: bool = field(default=False)
    mm_use_im_patch_token: bool = field(default=True)
    mm_patch_merge_type: Optional[str] = field(default='flat')
    mm_vision_select_feature: Optional[str] = field(default="patch")

@dataclass
class DataArguments:
    data_path: str = field(default=None,
                           metadata={"help": "Path to the training data."})
    lazy_preprocess: bool = False
    is_multimodal: bool = False
    image_folder: Optional[str] = field(default=None)
    image_aspect_ratio: str = 'square'

@dataclass
class TrainingArguments(transformers.TrainingArguments):
    cache_dir: Optional[str] = field(default=None)
    optim: str = field(default="adamw_torch")
    remove_unused_columns: bool = field(default=False)
    freeze_mm_mlp_adapter: bool = field(default=False)
    mpt_attn_impl: Optional[str] = field(default="triton")
    model_max_length: int = field(
        default=512,
        metadata={
            "help":
            "Maximum sequence length. Sequences will be right padded (and possibly truncated)."
        },
    )
    double_quant: bool = field(
        default=True,
        metadata={"help": "Compress the quantization statistics through double quantization."}
    )
    quant_type: str = field(
        default="nf4",
        metadata={"help": "Quantization data type to use. Should be one of `fp4` or `nf4`."}
    )
    bits: int = field(
        default=16,
        metadata={"help": "How many bits to use."}
    )
    lora_enable: bool = False
    lora_r: int = 64
    lora_alpha: int = 16
    lora_dropout: float = 0.05
    lora_weight_path: str = ""
    lora_bias: str = "none"
    mm_projector_lr: Optional[float] = None
    group_by_modality_length: bool = field(default=False)


[2025-12-28 10:14:53,862] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/ubuntu/miniconda3/envs/llava/lib/python3.10/site-packages/wandb/sdk/launch/builder/build.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/home/ubuntu/miniconda3/envs/llava/lib/python3.10/site-packages/wandb/sdk/launch/builder/build.py:11: UserWarning: Module wandb was already imported from /home/ubuntu/miniconda3/envs/llava/lib/python3.10/site-packages/wandb/__init__.py, but /home/ubuntu/LLaVA is being added to sys.path
  import pkg_resources


In [ ]:
from transformers import HfArgumentParser

args_dict = {
    #"deepspeed": "./scripts/zero2.json",
    "model_name_or_path": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    "version": "plain",
    "data_path": "/workspaces/LLaVA/CC3M_2.json",
    "image_folder": "/workspaces/LLaVA/images/",
    "vision_tower": "openai/clip-vit-large-patch14-336",
    "mm_projector_type": "mlp2x_gelu",
    "tune_mm_mlp_adapter": True,
    "mm_vision_select_layer": -2,
    "mm_use_im_start_end": False,
    "mm_use_im_patch_token": False,
    "bf16": True,
    "output_dir": "./checkpoints/Tinyllava-1.1B-pretrain",

    # TrainingArguments 相当
    "num_train_epochs": 1,
    "per_device_train_batch_size": 1,
    "per_device_eval_batch_size": 1,
    "gradient_accumulation_steps": 1,
    "evaluation_strategy": "no",
    "save_strategy": "steps",
    "save_steps": 1,
    "save_total_limit": 1,
    "learning_rate": 1e-3,
    "weight_decay": 0.0, # I don't know why 0.0
    "warmup_ratio": 0.03,
    "lr_scheduler_type": "cosine",
    "logging_steps": 1,
    "tf32": False, # switched from True for TinyLlama
    "model_max_length": 2048,
    "gradient_checkpointing": True,
    "dataloader_num_workers": 2,
    "lazy_preprocess": True,
    "report_to": "none",
}

In [ ]:
parser = HfArgumentParser((ModelArguments, DataArguments, TrainingArguments))
model_args, data_args, training_args = parser.parse_dict(args_dict)
print("model_args\n", model_args)
print("data_args\n", data_args)
print("training_args\n", training_args)

In [ ]:
# Model Constants
IGNORE_INDEX = -100
IMAGE_TOKEN_INDEX = -200
DEFAULT_IMAGE_TOKEN = "<image>"
DEFAULT_IMAGE_PATCH_TOKEN = "<im_patch>"
DEFAULT_IM_START_TOKEN = "<im_start>"
DEFAULT_IM_END_TOKEN = "<im_end>"
IMAGE_PLACEHOLDER = "<image-placeholder>"

In [ ]:
local_rank = training_args.local_rank
print("local_rank\n", local_rank)
compute_dtype = (torch.float16 if training_args.fp16 else (torch.bfloat16 if training_args.bf16 else torch.float32))
print("compute_dtype\n", compute_dtype)

In [ ]:
from transformers import CLIPVisionModel, CLIPImageProcessor, CLIPVisionConfig
import torch.nn as nn
# __init__
# load_model

# result = CLIPVisionTower(vision_tower, args=vision_tower_cfg, **kwargs)
class CLIPVisionTower(nn.Module):
    def __init__(self, vision_tower, args):
        # result = CLIPVisionTower(vision_tower, args=vision_tower_cfg, **kwargs)
        print("current file path", "llava/llava/model/multimodal_encoder/clip_encoder.py")
        print("def CLIPVisionTower.__init__(self, vision_tower, args")
        print("self\n", type(self))
        print("vision_tower\n", vision_tower) # openai/clip-vit-large-patch14-336
        print("args\n", args) # ModelArguments(model_name_or_path='TinyLlama/TinyLlama-1.1B-Chat-v1.0', version='plain', freeze_backbone=False, tune_mm_mlp_adapter=True, vision_tower='openai/clip-vit-large-patch14-336', mm_vision_select_layer=-2, pretrain_mm_mlp_adapter=None, mm_projector_type='mlp2x_gelu', mm_use_im_start_end=False, mm_use_im_patch_token=False, mm_patch_merge_type='flat', mm_vision_select_feature='patch')
        super().__init__()

        self.is_loaded = False

        print("self.is_loaded\n", self.is_loaded) # False

        self.vision_tower_name = vision_tower
        print("self.vision_tower_name\n", self.vision_tower_name) # openai/clip-vit-large-patch14-336
        self.select_layer = args.mm_vision_select_layer
        print("self.select_layer\n", self.select_layer) # -2
        self.select_feature = getattr(args, 'mm_vision_select_feature', 'patch')
        print("self.select_feature\n", self.select_feature) # patch

    def load_model(self):

        print("current file path", "llava/llava/model/multimodal_encoder/clip_encoder.py")
        print("def CLIPVisionTower.load_model(self)")
        print("self\n", type(self))
        print("self.vision_tower_name\n", self.vision_tower_name) # openai/clip-vit-large-patch14-336
        self.image_processor = CLIPImageProcessor.from_pretrained(self.vision_tower_name)
        print("self.image_processor\n", self.image_processor)
        """
        CLIPImageProcessor {
        "crop_size": {
            "height": 336,
            "width": 336
        },
        "do_center_crop": true,
        "do_convert_rgb": true,
        "do_normalize": true,
        "do_rescale": true,
        "do_resize": true,
        "feature_extractor_type": "CLIPFeatureExtractor",
        "image_mean": [
            0.48145466,
            0.4578275,
            0.40821073
        ],
        "image_processor_type": "CLIPImageProcessor",
        "image_std": [
            0.26862954,
            0.26130258,
            0.27577711
        ],
        "resample": 3,
        "rescale_factor": 0.00392156862745098,
        "size": {
            "shortest_edge": 336
        }
        }
        """
        self.vision_tower = CLIPVisionModel.from_pretrained(self.vision_tower_name)
        print("self.vision_tower\n", self.vision_tower)
        """
        CLIPVisionModel(
        (vision_model): CLIPVisionTransformer(
            (embeddings): CLIPVisionEmbeddings(
            (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
            (position_embedding): Embedding(577, 1024)
            )
            (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (encoder): CLIPEncoder(
            (layers): ModuleList(
                (0-23): 24 x CLIPEncoderLayer(
                (self_attn): CLIPAttention(
                    (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                    (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
                    (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
                    (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
                )
                (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
                (mlp): CLIPMLP(
                    (activation_fn): QuickGELUActivation()
                    (fc1): Linear(in_features=1024, out_features=4096, bias=True)
                    (fc2): Linear(in_features=4096, out_features=1024, bias=True)
                )
                (layer_norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
                )
            )
            )
            (post_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        )
        """
        self.vision_tower.requires_grad_(False)

        self.is_loaded = True
        print("self.is_loaded\n", self.is_loaded) # True

In [ ]:
print(model_args)

In [ ]:
vision_tower_name = "openai/clip-vit-large-patch14-336"
CLIPVisionTower_model = CLIPVisionTower(vision_tower_name, args=model_args)

In [ ]:
CLIPVisionTower_model.load_model()

In [ ]:
print(CLIPVisionTower_model)

In [ ]:
print(CLIPVisionTower_model.vision_tower)

In [ ]:
print(CLIPVisionTower_model.vision_tower.config)

In [ ]:
import os

def build_vision_tower(model_args, **kwargs):
    # vision_tower = build_vision_tower(model_args)
    print("current file path", "llava/llava/model/multimodal_encoder/builder.py")
    print("def build_vision_tower(model_args, **kwargs)")
    print("model_args\n", model_args) # ModelArguments(model_name_or_path='TinyLlama/TinyLlama-1.1B-Chat-v1.0', version='plain', freeze_backbone=False, tune_mm_mlp_adapter=True, vision_tower='openai/clip-vit-large-patch14-336', mm_vision_select_layer=-2, pretrain_mm_mlp_adapter=None, mm_projector_type='mlp2x_gelu', mm_use_im_start_end=False, mm_use_im_patch_token=False, mm_patch_merge_type='flat', mm_vision_select_feature='patch')
    print("kwargs\n", kwargs) # {}
    vision_tower = model_args.vision_tower
    print("vision_tower from model_args\n", vision_tower) # openai/clip-vit-large-patch14-336
    CLIPVisionTower_model = CLIPVisionTower(vision_tower, args=model_args, **kwargs)
    CLIPVisionTower_model.load_model()
    print("CLIPVisionTower_model\n", CLIPVisionTower_model)
    print("【EXIT】if is_absolute_path_exists or vision_tower.startswith('openai') or vision_tower.startswith('laion') or 'ShareGPT4V' in vision_tower:")
    return CLIPVisionTower_model

In [ ]:
build_vision_tower(model_args)

In [ ]:
import torch.nn as nn

def build_vision_projector(config, delay_load=False, **kwargs):
    # 2層MLPをハードコーディング（configからサイズのみ取得）
    return nn.Sequential(
        nn.Linear(config.mm_hidden_size, config.hidden_size),
        nn.GELU(),
        nn.Linear(config.hidden_size, config.hidden_size),
    )

In [ ]:
from transformers import AutoConfig

# 公式 LLaMA-2-7B の config をロード
TinyLlama_config = AutoConfig.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
TinyLlama_config.mm_hidden_size = CLIPVisionTower_model.vision_tower.config.hidden_size
print(TinyLlama_config)

In [ ]:
projector = build_vision_projector(TinyLlama_config)
print(projector)

In [ ]:
del(CLIPVisionTower_model)

In [ ]:
def print_mro(cls):
    print(f"MRO for {cls.__name__}:\n")
    for i, c in enumerate(cls.mro()):
        print(f"{i:2d}: {c.__module__}.{c.__name__}")
    print("\n\n")

In [ ]:
# LlavaBaseModel
# __init__
# get_vision_tower
# initialize_vision_modules
# unpad_image

class LlavaBaseModel:  # LLaMA, MPT などの全てのLLM用の共通機能をまとめたクラス

    # LlavaLlamaModelの__init__によって呼び出される
    def __init__(self):

        print("current file path", "llava/model/llava_arch.py")
        print("LlavaBaseModel.__init__(self, config)")
        # 「LlavaLlamaModelの__init__によって呼び出された時は」LlamaModelの__init_を呼び出す
        super(LlavaBaseModel, self).__init__()

In [ ]:
# transformers.models.llama.modeling_llama.LlamaModel
# URL: https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/modeling_llama.py
from transformers import LlamaModel

# LlavaBaseModelにLlamaの初期化クラスをくっつけたいので、LLamaModelをMROに追加。
# MROだと、LlavaLlamaModel > LlavaBaseModel > LlamaModel の順に解決される。
class LlavaLlamaModel(LlavaBaseModel, LlamaModel): 
    def __init__(self):

        print("current file path", "llava/llava/model/language_model/llava_llama.py")
        print("def LlavaLlamaModel.__init__(self, config: LlamaConfig)")
        print("self\n", type(self))
        # LlavaBaseModelの__init__を呼ぶ
        super(LlavaLlamaModel, self).__init__() 

In [ ]:
print_mro(LlavaBaseModel)
print_mro(LlavaLlamaModel)

In [ ]:
# LlavaBaseModel
# __init__
# get_vision_tower
# initialize_vision_modules
# unpad_image

class LlavaBaseModel:  # LLaMA, MPT などの全てのLLM用の共通機能をまとめたクラス

    # LlavaLlamaModelの__init__によって呼び出される
    def __init__(self, config):

        print("current file path", "llava/model/llava_arch.py")
        print("LlavaBaseModel.__init__(self, config)")
        print("config\n", config)
        # 「LlavaLlamaModelの__init__によって呼び出された時は」LlamaModelの__init_を呼び出す
        super(LlavaBaseModel, self).__init__(config)

In [ ]:
# transformers.models.llama.modeling_llama.LlamaModel
# URL: https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/modeling_llama.py
from transformers import LlamaConfig, LlamaModel

class LlavaLlamaConfig(LlamaConfig):
    model_type = "llava_llama"

# LlavaBaseModelにLlamaの初期化クラスをくっつけたいので、LLamaModelをMROに追加。
# MROだと、LlavaLlamaModel > LlavaBaseModel > LlamaModel の順に解決される。
class LlavaLlamaModel(LlavaBaseModel, LlamaModel): 
    config_class = LlavaLlamaConfig # from_pretrained の時にここが読みこまれる

    def __init__(self, config: LlamaConfig):

        print("current file path", "llava/llava/model/language_model/llava_llama.py")
        print("def LlavaLlamaModel.__init__(self, config: LlamaConfig)")
        print("self\n", type(self))
        print("config\n", config)
        # LlavaBaseModelの__init__を呼ぶ
        super(LlavaLlamaModel, self).__init__(config) 

In [ ]:
# LlavaBaseForCausalLM
# get_vision_tower
# encode_images
# prepare_inputs_labels_for_multimodal
# initialize_vision_tokenizer

# 以下のLlavaBaseForCausalLMのselfは、継承先のLlavaLlamaForCausalLMのselfが使用される

class LlavaBaseForCausalLM:

    def get_vision_tower(self):
        print("current file path", "llava/model/llava_arch.py")
        print("class LlavaBaseForCausalLM(ABC).get_vision_tower(self)")
        result = self.get_model().get_vision_tower()
        print("LlavaBaseForCausalLM(ABC).get_vision_tower(self) result (return)\n", result)
        """
        CLIPVisionTower(
        (vision_tower): CLIPVisionModel(
            (vision_model): CLIPVisionTransformer(
            (embeddings): CLIPVisionEmbeddings(
                (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
                (position_embedding): Embedding(577, 1024)
            )
            (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (encoder): CLIPEncoder(
                (layers): ModuleList(
                (0-23): 24 x CLIPEncoderLayer(
                    (self_attn): CLIPAttention(
                    (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                    (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
                    (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
                    (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
                    )
                    (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
                    (mlp): CLIPMLP(
                    (activation_fn): QuickGELUActivation()
                    (fc1): Linear(in_features=1024, out_features=4096, bias=True)
                    (fc2): Linear(in_features=4096, out_features=1024, bias=True)
                    )
                    (layer_norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
                )
                )
            )
            (post_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            )
        )
        )
        """
        return result

In [ ]:
from typing import List, Optional, Tuple, Union
from transformers.modeling_outputs import CausalLMOutputWithPast
from transformers import LlamaForCausalLM

class LlavaLlamaForCausalLM(LlamaForCausalLM, LlavaBaseForCausalLM):
    config_class = LlavaLlamaConfig # from_pretrainedの際にここが読み込まれる

    def __init__(self, config):

        print("current file path", "llava/llava/model/language_model/llava_llama.py")
        print("def LlavaLlamaForCausalLM.__init__(self, config)")
        print("self\n", type(self))
        print("config\n", config)
        super(LlamaForCausalLM, self).__init__(config)
        # LlamaForCausalLMのself.modelをLlavaLlamaModelに置き換える
        self.model = LlavaLlamaModel(config)
        print("self.model\n", self.model)
        # LlavaLlamaModelの初期化のあと、LlavaBaseModelの初期化も呼ばれる。
        self.pretraining_tp = config.pretraining_tp
        """
        self.model
        LlavaLlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
            (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
                (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
                (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
                (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
                (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
                (rotary_emb): LlamaRotaryEmbedding()
            )
            (mlp): LlamaMLP(
                (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
                (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
                (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
                (act_fn): SiLUActivation()
            )
            (input_layernorm): LlamaRMSNorm()
            (post_attention_layernorm): LlamaRMSNorm()
            )
        )
        (norm): LlamaRMSNorm()
        )
        """
        #print("self.pretraining_tp\n", self.pretraining_tp) # 1
        #print("self.vocab_size\n", self.vocab_size) # 32_000
        #print("self.lm_head\n", self.lm_head) # Linear(in_features=4096, out_features=32000, bias=False)

        # Initialize weights and apply final processing
        #self.post_init()

In [ ]:
"""
from transformers import AutoConfig

# まず config.json をロードして Config クラスを自動判別
config = AutoConfig.from_pretrained(
    model_args.model_name_or_path,
    cache_dir=training_args.cache_dir
)

print("model_args.model_name_or_path\n", model_args.model_name_or_path)
print("training_args.cache_dir\n", training_args.cache_dir)
print("")
print("Loaded config:\n", config)
"""

In [ ]:
print_mro(LlavaBaseForCausalLM)
print_mro(LlavaLlamaForCausalLM)

In [ ]:
LlavaLlamaForCausalLM_model = LlavaLlamaForCausalLM.from_pretrained(model_args.model_name_or_path)

In [ ]:
print("LlavaLlamaForCausalLM_model\n", LlavaLlamaForCausalLM_model)

In [ ]:
LlavaLlamaForCausalLM_model.enable_input_require_grads()

In [ ]:
import dataclasses
from typing import List
from enum import auto, Enum

class SeparatorStyle(Enum):
    """Different separator style."""
    SINGLE = auto()
    TWO = auto()
    MPT = auto()
    PLAIN = auto()
    LLAMA_2 = auto()


@dataclasses.dataclass
class Conversation:
    """A class that keeps all conversation history."""
    system: str
    roles: List[str]
    messages: List[List[str]]
    offset: int
    sep_style: SeparatorStyle = SeparatorStyle.SINGLE
    sep: str = "###"
    sep2: str = None
    version: str = "Unknown"
    skip_next: bool = False


conv_llava_plain = Conversation(
    system="",
    roles=("", ""),
    messages=(
    ),
    offset=0,
    sep_style=SeparatorStyle.PLAIN,
    sep="\n",
)


conv_templates = {
    "plain": conv_llava_plain,
}

In [ ]:
"""
import inspect
print(inspect.getattr_static(LlamaForCausalLM, "from_pretrained"))
print(inspect.getattr_static(LlamaForCausalLM, "enable_input_require_grads"))
"""

In [ ]:
print(model_args.model_name_or_path)
print(training_args.cache_dir)
print(training_args.model_max_length)

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_args.model_name_or_path,
    cache_dir=training_args.cache_dir,
    model_max_length=training_args.model_max_length, 
    padding_side="right", # 実際にパディングを行うのは tokenizer か DataCollator のどちらかだが、padding_side 自体はtokenizer側で行う
    use_fast=False,
)

In [ ]:
tokenizer

In [ ]:
print("pad_token:", tokenizer.pad_token)
print("pad_token_id:", tokenizer.pad_token_id)
print("unk_token:", tokenizer.unk_token) # unkownの略です。類似の単語と間違えないでください。
print("unk_token_id:", tokenizer.unk_token_id)
print("tokenizer\n", tokenizer)

In [ ]:
tokenizer.pad_token = tokenizer.unk_token

In [ ]:
print("pad_token:", tokenizer.pad_token)
print("pad_token_id:", tokenizer.pad_token_id)
print("unk_token:", tokenizer.unk_token)
print("unk_token_id:", tokenizer.unk_token_id)
print("tokenizer\n", tokenizer)

In [ ]:
print("model_args.version\n", model_args.version)
default_conversation = conv_templates[model_args.version]
print("default_conversation\n", default_conversation)

In [ ]:
print("model_args.vision_tower\n", model_args.vision_tower)

In [ ]:
# LlavaLlamaForCausalLMの関数。LlavaLlamaModelをgetする
def get_model(self):

    print("current file path", "llava/llava/model/language_model/llava_llama.py")
    print("def LlavaLlamaForCausalLM.get_model(self)")
    print("self\n", type(self))
    print("self.model (return)\n", self.model)
    return self.model

In [ ]:
LlavaLlamaForCausalLM.get_model = get_model

In [ ]:
LlavaLlamaModel_model = LlavaLlamaForCausalLM_model.get_model()

In [ ]:
def config(self):

    print("current file path", "llava/llava/model/multimodal_encoder/clip_encoder.py")
    print("def CLIPVisionTower.config(self)")
    print("self\n", type(self))
    print("self.is_loaded\n", self.is_loaded) # True
    print(f"【COND】 is_loaded={self.is_loaded}")
    if self.is_loaded:
        # 【ENTER】
        print("【ENTER】if self.is_loaded:")
        result = self.vision_tower.config
        print("result (return)\n", type(result))
        print("【EXIT】if self.is_loaded:")
    else:
      pass
    print("result (return)\n", result)
    """
    CLIPVisionConfig {
    "_name_or_path": "openai/clip-vit-large-patch14-336",
    "attention_dropout": 0.0,
    "dropout": 0.0,
    "hidden_act": "quick_gelu",
    "hidden_size": 1024,
    "image_size": 336,
    "initializer_factor": 1.0,
    "initializer_range": 0.02,
    "intermediate_size": 4096,
    "layer_norm_eps": 1e-05,
    "model_type": "clip_vision_model",
    "num_attention_heads": 16,
    "num_channels": 3,
    "num_hidden_layers": 24,
    "patch_size": 14,
    "projection_dim": 768,
    "transformers_version": "4.31.0"
    }
    """
    return result

In [ ]:
CLIPVisionTower.config = property(config)

In [ ]:
def initialize_vision_modules(self, model_args, fsdp=None):

  print("current file path", "llava/model/llava_arch.py")
  print("def initialize_vision_modules(self, model_args, fsdp=None)")
  print("model_args\n", model_args) #  ModelArguments(model_name_or_path='TinyLlama/TinyLlama-1.1B-Chat-v1.0', version='plain', freeze_backbone=False, tune_mm_mlp_adapter=True, vision_tower='openai/clip-vit-large-patch14-336', mm_vision_select_layer=-2, pretrain_mm_mlp_adapter=None, mm_projector_type='mlp2x_gelu', mm_use_im_start_end=False, mm_use_im_patch_token=False, mm_patch_merge_type='flat', mm_vision_select_feature='patch')
  print("fsdp\n", fsdp) # []
  vision_tower = model_args.vision_tower
  print("vision_tower from model_args\n", vision_tower) # openai/clip-vit-large-patch14-336
  mm_vision_select_layer = model_args.mm_vision_select_layer
  print("mm_vision_select_layer from model_args\n", mm_vision_select_layer) # -2
  mm_vision_select_feature = model_args.mm_vision_select_feature
  print("mm_vision_select_feature from model_args\n", mm_vision_select_feature) # patch
  pretrain_mm_mlp_adapter = model_args.pretrain_mm_mlp_adapter
  print("pretrain_mm_mlp_adapter from model_args\n", pretrain_mm_mlp_adapter) # None
  mm_patch_merge_type = model_args.mm_patch_merge_type
  # 下記はself.config.mm_vision_towerに関するもの。self.vision_towerは依然としてNone
  self.config.mm_vision_tower = vision_tower
  print("self.config.mm_vision_tower\n", self.config.mm_vision_tower) # None

  print("【COND】 self.get_vision_tower()\n", self.get_vision_tower()) # None
  print(f"【COND】 get_vision_tower_is_None={self.get_vision_tower() is None}")
  if self.get_vision_tower() is None:
      #【ENTER】self.vision_tower, self.get_vision_towerはNoneなのでこの分岐に入る。
      print("【ENTER】if self.get_vision_tower() is None:")
      print("[ENTER] self.get_vision_tower() is None")
      # build_vision_tower(model_args) はちょっと奥の依存関係が深い
      vision_tower = build_vision_tower(model_args)
      print("vision_tower after build_vision_tower\n", vision_tower)
      """
      CLIPVisionTower(
      (vision_tower): CLIPVisionModel(
      (vision_model): CLIPVisionTransformer(
          (embeddings): CLIPVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
          (position_embedding): Embedding(577, 1024)
          )
          (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (encoder): CLIPEncoder(
          (layers): ModuleList(
              (0-23): 24 x CLIPEncoderLayer(
              (self_attn): CLIPAttention(
                  (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                  (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
                  (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
                  (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
              )
              (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
              (mlp): CLIPMLP(
                  (activation_fn): QuickGELUActivation()
                  (fc1): Linear(in_features=1024, out_features=4096, bias=True)
                  (fc2): Linear(in_features=4096, out_features=1024, bias=True)
              )
              (layer_norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
              )
          )
          )
          (post_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
      )
      )
      """
      # 分散学習(FSDP)を使うかどうか. 今回は [] 空のリストとなるので、Noneではないが、len(fsdp) == 0
      print("【COND】 fsdp\n", fsdp) # []
      print(f"【COND】 fsdp_is_not_None={fsdp is not None} len_fsdp={len(fsdp) if fsdp is not None else 'N/A'}") # fsdp_is_not_None=True len_fsdp=0
      if fsdp is not None and len(fsdp) > 0:
        pass
      else:
          # 【ENTER】else of if fsdp is not None and len(fsdp) > 0:
          print("【COND】 else_fsdp_is_not_None_and_len_fsdp_gt_0=True")
          print("【ENTER】else of if fsdp is not None and len(fsdp) > 0:")
          self.vision_tower = vision_tower
          print("self.vision_tower\n", self.vision_tower)
          """
          CLIPVisionTower(
          (vision_tower): CLIPVisionModel(
              (vision_model): CLIPVisionTransformer(
              (embeddings): CLIPVisionEmbeddings(
                  (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
                  (position_embedding): Embedding(577, 1024)
              )
              (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
              (encoder): CLIPEncoder(
                  (layers): ModuleList(
                  (0-23): 24 x CLIPEncoderLayer(
                      (self_attn): CLIPAttention(
                      (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                      (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
                      (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
                      (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
                      )
                      (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
                      (mlp): CLIPMLP(
                      (activation_fn): QuickGELUActivation()
                      (fc1): Linear(in_features=1024, out_features=4096, bias=True)
                      (fc2): Linear(in_features=4096, out_features=1024, bias=True)
                      )
                      (layer_norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
                  )
                  )
              )
              (post_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
              )
          )
          )
          """
          print("【EXIT】else of if fsdp is not None and len(fsdp) > 0:")

      print("【EXIT】if self.get_vision_tower() is None:")
  else:
    pass

  self.config.use_mm_proj = True
  print("self.config.use_mm_proj set to True") # True
  self.config.mm_projector_type = getattr(model_args, 'mm_projector_type', 'linear')
  print("self.config.mm_projector_type\n", self.config.mm_projector_type) # mlp2x_gelu
  self.config.mm_hidden_size = vision_tower.config.hidden_size
  print("self.config.mm_hidden_size\n", self.config.mm_hidden_size) # 1024
  self.config.mm_vision_select_layer = mm_vision_select_layer
  print("self.config.mm_vision_select_layer\n", self.config.mm_vision_select_layer) # -2
  self.config.mm_vision_select_feature = mm_vision_select_feature
  print("self.config.mm_vision_select_feature\n", self.config.mm_vision_select_feature) # patch
  self.config.mm_patch_merge_type = mm_patch_merge_type
  print("self.config.mm_patch_merge_type\n", self.config.mm_patch_merge_type) # flat

  # mm_projector_is_None=True
  print(f"【COND】 mm_projector_is_None={getattr(self, 'mm_projector', None) is None}")
  if getattr(self, 'mm_projector', None) is None:
      # 【ENTER】
      print("【ENTER】if getattr(self, 'mm_projector', None) is None:")
      self.mm_projector = build_vision_projector(self.config)
      """
      Sequential(
        (0): Linear(in_features=1024, out_features=2048, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=2048, out_features=2048, bias=True)
      )
      """
      print("self.mm_projector after build_vision_projector\n", self.mm_projector)
      print("mm_patch_merge_type\n", mm_patch_merge_type) # flat
      print(f"【COND】 unpad_in_mm_patch_merge_type={'unpad' in mm_patch_merge_type}")
      if 'unpad' in mm_patch_merge_type:
        pass
      print("【EXIT】if getattr(self, 'mm_projector', None) is None:")
  else:
    pass

  print(f"【COND】 pretrain_mm_mlp_adapter_is_not_None={pretrain_mm_mlp_adapter is not None}")
  if pretrain_mm_mlp_adapter is not None:
    pass

In [ ]:
LlavaBaseModel.initialize_vision_modules = initialize_vision_modules

In [ ]:
def get_vision_tower(self):

    print("current file path", "llava/model/llava_arch.py")
    print("def get_vision_tower(self)")
    vision_tower = getattr(self, 'vision_tower', None)
    print("vision_tower (raw)\n", vision_tower)
    """
    CLIPVisionTower(
    (vision_tower): CLIPVisionModel(
        (vision_model): CLIPVisionTransformer(
        (embeddings): CLIPVisionEmbeddings(
            (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
            (position_embedding): Embedding(577, 1024)
        )
        (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (encoder): CLIPEncoder(
            (layers): ModuleList(
            (0-23): 24 x CLIPEncoderLayer(
                (self_attn): CLIPAttention(
                (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
                )
                (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
                (mlp): CLIPMLP(
                (activation_fn): QuickGELUActivation()
                (fc1): Linear(in_features=1024, out_features=4096, bias=True)
                (fc2): Linear(in_features=4096, out_features=1024, bias=True)
                )
                (layer_norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            )
            )
        )
        (post_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
    )
    )
    """
    print("type(vision_tower)\n", type(vision_tower))
    print(f"【COND】 type_vision_tower_is_list={type(vision_tower) is list}")  # False
    if type(vision_tower) is list:
        # 【SKIP】
        print("【ENTER】if type(vision_tower) is list:")
        vision_tower = vision_tower[0]
        print("【EXIT】if type(vision_tower) is list:")
    print("vision_tower (return)\n", vision_tower)
    """
    vision_tower (return)
    CLIPVisionTower(
    (vision_tower): CLIPVisionModel(
        (vision_model): CLIPVisionTransformer(
        (embeddings): CLIPVisionEmbeddings(
            (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
            (position_embedding): Embedding(577, 1024)
        )
        (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (encoder): CLIPEncoder(
            (layers): ModuleList(
            (0-23): 24 x CLIPEncoderLayer(
                (self_attn): CLIPAttention(
                (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
                )
                (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
                (mlp): CLIPMLP(
                (activation_fn): QuickGELUActivation()
                (fc1): Linear(in_features=1024, out_features=4096, bias=True)
                (fc2): Linear(in_features=4096, out_features=1024, bias=True)
                )
                (layer_norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            )
            )
        )
        (post_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
    )
    )
    """
    return vision_tower

In [ ]:
LlavaBaseModel.get_vision_tower = get_vision_tower

In [ ]:
LlavaLlamaModel_model.initialize_vision_modules(
    model_args=model_args,
    fsdp=training_args.fsdp
)

In [ ]:
vision_tower = LlavaLlamaForCausalLM_model.get_vision_tower()
print("vision_tower\n", vision_tower)
vision_tower.to(dtype=torch.bfloat16 if training_args.bf16 else torch.float16, device=training_args.device)

data_args.image_processor = vision_tower.image_processor
print("data_args.image_processor\n", data_args.image_processor)
data_args.is_multimodal = True
print("data_args.is_multimodal\n", data_args.is_multimodal) # True

LlavaLlamaForCausalLM_model.config.image_aspect_ratio = data_args.image_aspect_ratio
print("model.config.image_aspect_ratio\n", LlavaLlamaForCausalLM_model.config.image_aspect_ratio) # square
LlavaLlamaForCausalLM_model.config.tokenizer_padding_side = tokenizer.padding_side
print("model.config.tokenizer_padding_side\n", LlavaLlamaForCausalLM_model.config.tokenizer_padding_side) # right
LlavaLlamaForCausalLM_model.config.tokenizer_model_max_length = tokenizer.model_max_length
print("model.config.tokenizer_model_max_length\n", LlavaLlamaForCausalLM_model.config.tokenizer_model_max_length) # 2048

In [ ]:
LlavaLlamaForCausalLM_model.config.tune_mm_mlp_adapter = model_args.tune_mm_mlp_adapter
training_args.tune_mm_mlp_adapter = model_args.tune_mm_mlp_adapter
print(f"【COND】 tune_mm_mlp_adapter={model_args.tune_mm_mlp_adapter}") # True
if model_args.tune_mm_mlp_adapter:
    # 【ENTER】 tune_mm_mlp_adapter=True なので、この分岐に入る
    print("【ENTER】if model_args.tune_mm_mlp_adapter:")
    # モデル全体の全パラメータを「学習不可（requires_grad=False）」にする
    # これで通常の重みは全て凍結される
    LlavaLlamaForCausalLM_model.requires_grad_(False)
    for p in LlavaLlamaForCausalLM_model.get_model().mm_projector.parameters():
        # mm_projector（画像特徴量→テキスト特徴量への変換層）の全パラメータだけを「学習可能（requires_grad=True）」に戻す
        # これで mm_projector のみ学習されることになる
        print("model.get_model().mm_projector.parameters()", LlavaLlamaForCausalLM_model.get_model().mm_projector.parameters())
        p.requires_grad = True
    print("【EXIT】if model_args.tune_mm_mlp_adapter:")

In [ ]:
LlavaLlamaForCausalLM_model.config.mm_use_im_start_end = model_args.mm_use_im_start_end
data_args.mm_use_im_start_end = model_args.mm_use_im_start_end
training_args.mm_use_im_start_end = model_args.mm_use_im_start_end

LlavaLlamaForCausalLM_model.config.mm_projector_lr = training_args.mm_projector_lr
print("training_args.mm_projector_lr", training_args.mm_projector_lr)

LlavaLlamaForCausalLM_model.config.mm_use_im_patch_token = model_args.mm_use_im_patch_token
print("model_args.mm_use_im_patch_token", model_args.mm_use_im_patch_token)
#LlavaLlamaForCausalLM_model.initialize_vision_tokenizer(model_args, tokenizer=tokenizer)
#print("【EXIT】if model_args.vision_tower is not None:")

In [ ]:
def rank0_print(*args):

    print("current file path", "llava/train/train.py")
    print("def rank0_print(*args)")
    print("args\n", args) # ('Formatting inputs...Skip in lazy mode',)
    if local_rank == 0:
        print(*args)

In [ ]:
from torch.utils.data import Dataset
import json

# URL: https://github.com/pytorch/pytorch/blob/main/torch/utils/data/dataset.py
class LazySupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str,
                 tokenizer: transformers.PreTrainedTokenizer,
                 data_args: DataArguments):

        print("current file path", "llava/train/train.py")
        print("def LazySupervisedDataset.__init__(self, data_path, tokenizer, data_args)")
        print("data_path\n", data_path) # /content/LLaVA/blip_laion_cc_sbu_1.json
        print("tokenizer\n", type(tokenizer)) # <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>
        print("data_args\n", data_args) # DataArguments(data_path='/content/LLaVA/blip_laion_cc_sbu_1.json', lazy_preprocess=True, is_multimodal=True, image_folder='/content/LLaVA/images', image_aspect_ratio='square')
        super(LazySupervisedDataset, self).__init__()
        list_data_dict = json.load(open(data_path, "r"))
        # 今回は1サンプルだけなのでprintしても危険ではない
        print("list_data_dict", list_data_dict)

        rank0_print("Formatting inputs...Skip in lazy mode") # Formatting inputs...Skip in lazy mode
        self.tokenizer = tokenizer
        print("self.tokenizer\n", self.tokenizer)
        self.list_data_dict = list_data_dict
        print("self.list_data_dict\n", self.list_data_dict)
        self.data_args = data_args
        print("self.data_args\n", self.data_args)
    
    def __len__(self):

        print("current file path", "llava/train/train.py")
        print("def LazySupervisedDataset.__len__(self)")
        return len(self.list_data_dict)    

In [ ]:
print_mro(LazySupervisedDataset)

In [ ]:
# だいぶ間が空いているので復習
"""
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_args.model_name_or_path,
    cache_dir=training_args.cache_dir,
    model_max_length=training_args.model_max_length,
    padding_side="right",
    use_fast=False,
)
"""

print("tokenizer\n", tokenizer)

In [ ]:
train_dataset = LazySupervisedDataset(data_path=data_args.data_path, tokenizer=tokenizer, data_args=data_args)

In [ ]:
train_dataset.list_data_dict[0]

In [ ]:
i = 0
data_sample = train_dataset.list_data_dict[i]
print("data_sample\n", data_sample)

In [ ]:
if 'image' in data_sample:
    image_file = data_sample['image']
    print("image_file\n", image_file)
    image_folder = train_dataset.data_args.image_folder
    print("image_folder\n", image_folder)
    image_path = os.path.join(image_folder, image_file)
    print("image_path\n", image_path)

In [ ]:
image_processor = train_dataset.data_args.image_processor
print("image_processor\n", image_processor)

In [ ]:
from PIL import Image
image = Image.open(image_path).convert('RGB')
image

In [ ]:
# URL: https://github.com/huggingface/transformers/blob/main/src/transformers/models/clip/image_processing_clip.py
image = image_processor.preprocess(image, return_tensors='pt')['pixel_values'][0]
print("image_tensor_shape\n", image.shape)
image

In [ ]:
i = 0
data_sample = train_dataset.list_data_dict[i]

if isinstance(i, int):
    data_sample_list = [data_sample]
if 'image' in data_sample:
    image_file = data_sample['image']
    image_folder = train_dataset.data_args.image_folder
    image_path = os.path.join(image_folder, image_file)
    image_processor = train_dataset.data_args.image_processor
    try:
        image = Image.open(image_path).convert('RGB')
    except Exception as e:
        print(f"Failed to load image {image_path}: {e}")
        raise e

    image = image_processor.preprocess(image, return_tensors='pt')['pixel_values'][0]

In [ ]:
# i = 0 
import copy
dialog_pair = copy.deepcopy(data_sample["conversations"])
print("dialog_pair\n", dialog_pair)
# __getitem__ では 1サンプルずつ取り出すので、基本的にリストの長さは1
# pair は1組だが、リストにした方が便利な時があるのでリスト形式に変換する。
# リストといっても、リスト形式にしたという意味で、複数サンプルがあるわけではないので注意
# dialog_pair 自体がリストなので、二重リストになる。
dialog_pair_list = [dialog_pair]
print("dialog_pair_list\n", dialog_pair_list)

In [ ]:
# 見づらいので、もう一度見やすく定義する
dialog_pair_list = [[
    {'from': 'human', 'value': 'Render a clear and concise summary of the photo.\n<image>'},
    {'from': 'gpt', 'value': 'illustration of a summer holiday in bright colors .'}
]]

print("len(dialog_pair_list)\n", len(dialog_pair_list))

In [ ]:
print("data_args\n", data_args)

In [ ]:
is_multimodal = data_args.is_multimodal
print("is_multimodal\n", is_multimodal)

In [ ]:
dialog_pair = dialog_pair_list[0]
print("pair current loop\n", dialog_pair)

In [ ]:
human_dialog = dialog_pair[0]

In [ ]:
print("sentence['value']\n", human_dialog['value'])
print("\nDEFAULT_IMAGE_TOKEN\n", DEFAULT_IMAGE_TOKEN)
print("【COND】 if DEFAULT_IMAGE_TOKEN in sentence['value']:", DEFAULT_IMAGE_TOKEN in human_dialog['value'])

In [ ]:
# strip() は先頭と末尾の空白文字（スペース、タブ、改行など）をすべて削除するPythonの標準メソッドです。
# repleace で <image> (DEFAULT_IMAGE_TOKEN) を空文字に置換したあとに、strip()を適用して削除する。

human_dialog['value'] = human_dialog['value'].replace(DEFAULT_IMAGE_TOKEN, '').strip()
print("human_dialog['value'] after remove DEFAULT_IMAGE_TOKEN\n", human_dialog['value'])
print("===")

human_dialog['value'] = DEFAULT_IMAGE_TOKEN + '\n' + human_dialog['value']
print("human_dialog['value'] after adding DEFAULT_IMAGE_TOKEN to the front\n", human_dialog['value'])
print("===")

human_dialog['value'] = human_dialog['value'].strip()
print("human_dialog['value'] after strip()\n", human_dialog['value'])
print("===")

In [ ]:
print("human_dialog['value']\n", human_dialog['value'])
print("\nDEFAULT_IMAGE_TOKEN\n", DEFAULT_IMAGE_TOKEN)
print("【COND】 if DEFAULT_IMAGE_TOKEN in human_dialog['value']:", DEFAULT_IMAGE_TOKEN in human_dialog['value'])
if DEFAULT_IMAGE_TOKEN in human_dialog['value']: # False
    # 【SKIP】
    human_dialog['value'] = human_dialog['value'].replace(DEFAULT_IMAGE_TOKEN, '').strip()
    print("human_dialog['value'] after remove DEFAULT_IMAGE_TOKEN\n", human_dialog['value'])
    print("===")

    human_dialog['value'] = DEFAULT_IMAGE_TOKEN + '\n' + human_dialog['value']
    print("human_dialog['value'] after adding DEFAULT_IMAGE_TOKEN to the front\n", human_dialog['value'])
    print("===")

    human_dialog['value'] = human_dialog['value'].strip()
    print("human_dialog['value'] after strip()\n", human_dialog['value'])

print("DONE")

In [ ]:
gpt_dialog = dialog_pair[1]
print("gpt_dialog current loop\n", gpt_dialog)

In [ ]:
print("gpt_dialog['value']\n", gpt_dialog['value'])
print("\nDEFAULT_IMAGE_TOKEN\n", DEFAULT_IMAGE_TOKEN)
print("【COND】 if DEFAULT_IMAGE_TOKEN in gpt_dialog['value']:", DEFAULT_IMAGE_TOKEN in gpt_dialog['value'])
if DEFAULT_IMAGE_TOKEN in gpt_dialog['value']: # False
    # 【SKIP】
    gpt_dialog['value'] = gpt_dialog['value'].replace(DEFAULT_IMAGE_TOKEN, '').strip()
    print("gpt_dialog['value'] after remove DEFAULT_IMAGE_TOKEN\n", gpt_dialog['value'])
    print("===")

    gpt_dialog['value'] = DEFAULT_IMAGE_TOKEN + '\n' + gpt_dialog['value']
    print("gpt_dialog['value'] after adding DEFAULT_IMAGE_TOKEN\n", gpt_dialog['value'])
    print("===")

    gpt_dialog['value'] = gpt_dialog['value'].strip()
    print("gpt_dialog['value'] after strip()\n", gpt_dialog['value'])

print("DONE")

In [ ]:
from typing import Sequence
from typing import Dict

def preprocess_multimodal(
    dialog_pair: Sequence[str],
    data_args: DataArguments
) -> Dict:

    print("current file path", "llava/train/train.py")
    print("def preprocess_multimodal(dialog_pair, data_args)")
    print("dialog_pair\n", dialog_pair) # [{'from': 'human', 'value': 'Give a brief description of the image.\n<image>'}, {'from': 'gpt', 'value': 'the divine queen in her elaborate masks canvas print featuring the face and hands of a woman with red hair'}]
    print("data_args\n", data_args) # DataArguments(data_path='/content/LLaVA/blip_laion_cc_sbu_1.json', lazy_preprocess=True, is_multimodal=True, image_folder='/content/LLaVA/images', image_aspect_ratio='square')
    for dialog in dialog_pair:
        if DEFAULT_IMAGE_TOKEN in dialog['value']:
            dialog['value'] = dialog['value'].replace(DEFAULT_IMAGE_TOKEN, '').strip()
            dialog['value'] = DEFAULT_IMAGE_TOKEN + '\n' + dialog['value']
            dialog['value'] = dialog['value'].strip()
    return dialog_pair

In [ ]:
dialog_pair= [
    {'from': 'human', 'value': 'Render a clear and concise summary of the photo.\n<image>'},
    {'from': 'gpt', 'value': 'illustration of a summer holiday in bright colors .'}
]

In [ ]:
dialog_pair = preprocess_multimodal(dialog_pair, data_args)
print("dialog_pair\n", dialog_pair)

In [ ]:
# 他のサンプルでも試してみる
dialog_pair = [
    {'from': 'human', 'value': '<image>\nShare a concise interpretation of the image provided.'},
    {'from': 'gpt', 'value': 'water pollution in the city'}
]

In [ ]:
dialog_pair = preprocess_multimodal(dialog_pair, data_args)
print("dialog_pair\n", dialog_pair)

In [ ]:
""""
def __len__(self):

    print("current file path", "llava/train/train.py")
    print("def LazySupervisedDataset.__len__(self)")
    return len(self.list_data_dict)
LazySupervisedDataset.__len__ = __len__
"""

In [ ]:
prompt = "<image>illustration of a summer holiday in bright colors . \n"
image_token_index = IMAGE_TOKEN_INDEX # -200
return_tensors = "pt"

from transformers import AutoTokenizer
# 例: TinyLlama用
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", use_fast=False)
tokenizer.pad_token = tokenizer.unk_token

print("prompt:", prompt)
print("tokenizer:", tokenizer)
print("image_token_index:", image_token_index)
print("return_tensors:", return_tensors)

In [ ]:
text_before_image, text_after_image = prompt.split("<image>")
print("text_before_image\n", text_before_image) # Nothing
print("text_after_image\n", text_after_image) # caption

In [ ]:
# tokenizer(chunk).input_ids の 1 は、<s> という、BOS (Beginning Of Sentence) トークンのIDです。
# 全く何もない場合でも、<s> という 特殊トークンとして扱われます。
ids_before_image = tokenizer(text_before_image).input_ids
print("ids_before_image\n", ids_before_image)
ids_after_image = tokenizer(text_after_image).input_ids
print("ids_after_image\n", ids_after_image)

# [テキスト前] + [画像トークン] + [テキスト後]
ids_after_image_no_bos = ids_after_image[1:]
input_ids = ids_before_image + [image_token_index] + ids_after_image_no_bos
print("input_ids\n", input_ids)

In [ ]:
if return_tensors == "pt":
    tokenized_ids = torch.tensor(input_ids, dtype=torch.long)

print("tokenized_ids\n", tokenized_ids)

In [ ]:
def tokenizer_image_token(prompt, tokenizer, image_token_index=IMAGE_TOKEN_INDEX, return_tensors=None):
    # <image> の前後で分割
    text_before_image, text_after_image = prompt.split("<image>")

    # トークナイズ
    ids_before_image = tokenizer(text_before_image).input_ids
    ids_after_image = tokenizer(text_after_image).input_ids
    ids_after_image_no_bos = ids_after_image[1:]

    # [テキスト前] + [画像トークン] + [テキスト後]
    input_ids = ids_before_image + [image_token_index] + ids_after_image_no_bos

    if return_tensors == "pt":
        return torch.tensor(input_ids, dtype=torch.long)
    return input_ids


In [ ]:
prompt = "<image>illustration of a summer holiday in bright colors .\n'"
image_token_index = IMAGE_TOKEN_INDEX # -200
return_tensors = "pt"

from transformers import AutoTokenizer
# 例: TinyLlama用
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", use_fast=False)
tokenizer.pad_token = tokenizer.unk_token

print("prompt:", prompt)
print("tokenizer:", tokenizer)
print("image_token_index:", image_token_index)
print("return_tensors:", return_tensors)

tokenizer_image_token(prompt, tokenizer, image_token_index, return_tensors)

In [ ]:
"""
# tokenizer(chunk).input_ids の 1 は、<s> という、BOS (Beginning Of Sentence) トークンのIDです。
# 全く何もない場合でも、<s> という 特殊トークンとして扱われます。
for chunk in prompt.split("<image>"):
    print("chunk\n", chunk)
    print("tokenizer(chunk)\n", tokenizer(chunk))
    print("tokenizer(chunk).input_ids\n", tokenizer(chunk).input_ids)
    print("")

# セル2：<image>で分割してトークン化
prompt_chunks = [tokenizer(chunk).input_ids for chunk in prompt.split("<image>")]
print("prompt_chunks =", prompt_chunks)

print("prompt_chunks\n", prompt_chunks)
print("prompt_chunks[0]\n", prompt_chunks[0])
print("prompt_chunks[0][0]\n", prompt_chunks[0][0])

# ======================
# セル3：画像トークンを間に挟む処理（関数なし）
# ======================
input_ids = []
offset = 0

# 最初のチャンクがBOSトークンで始まっていたら、その分をスキップ
if len(prompt_chunks) > 0 and len(prompt_chunks[0]) > 0 and prompt_chunks[0][0] == tokenizer.bos_token_id:
    offset = 1
    input_ids.append(prompt_chunks[0][0])
print("offset =", offset)
print("input_ids\n", input_ids)

# チャンクと画像トークンを交互に並べる
for i, chunk in enumerate(prompt_chunks):
    # チャンクの実体を追加
    print("current chunk\n", chunk)
    input_ids.extend(chunk[offset:])
    print("input_ids\n", input_ids)
    # 最後以外には画像トークンを挿入
    if i < len(prompt_chunks) - 1: # len(prompt_chunks) -1  = 2 -1 = 1 より、i=0
        print("i", i)
        input_ids.extend([image_token_index])
        print(f"--> insert image_token_index ({image_token_index}) between chunks")

print("統合後の input_ids:", input_ids)


# ======================
# セル4：return_tensors の処理
# ======================
import torch

if return_tensors == "pt":
    input_ids = torch.tensor(input_ids, dtype=torch.long)

print("最終出力:")
print(input_ids)

# セル4：offset処理と最初の BOS トークン確認
input_ids = []
offset = 0

if len(prompt_chunks) > 0 and len(prompt_chunks[0]) > 0 and prompt_chunks[0][0] == tokenizer.bos_token_id:
    offset = 1
    input_ids.append(prompt_chunks[0][0])

print("offset =", offset)
print("初期input_ids =", input_ids)

# セル6：return_tensorsの処理
import torch

if return_tensors is not None:
    if return_tensors == "pt":
        input_ids = torch.tensor(input_ids, dtype=torch.long)
    else:
        raise ValueError(f"Unsupported tensor type: {return_tensors}")

print("最終出力:")
print(input_ids)
"""


In [ ]:
"""
def tokenizer_image_token(prompt, tokenizer, image_token_index=IMAGE_TOKEN_INDEX, return_tensors=None):

    print("current file path", "llava/mm_utils.py")
    print("def tokenizer_image_token(prompt, tokenizer, image_token_index=IMAGE_TOKEN_INDEX, return_tensors=None)")
    print("prompt\n", prompt) # <image>the divine queen in her elaborate masks canvas print featuring the face and hands of a woman with red hair
    print("tokenizer\n", tokenizer) #  LlamaTokenizer(name_or_path='TinyLlama/TinyLlama-1.1B-Chat-v1.0', vocab_size=32000, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False)
    print("image_token_index\n", image_token_index) # -200
    print("return_tensors\n", return_tensors) # pt
    prompt_chunks = [tokenizer(chunk).input_ids for chunk in prompt.split('<image>')]

    def insert_separator(X, sep):
        return [ele for sublist in zip(X, [sep]*len(X)) for ele in sublist][:-1]

    input_ids = []
    offset = 0
    if len(prompt_chunks) > 0 and len(prompt_chunks[0]) > 0 and prompt_chunks[0][0] == tokenizer.bos_token_id:
        offset = 1
        input_ids.append(prompt_chunks[0][0])

    for x in insert_separator(prompt_chunks, [image_token_index] * (offset + 1)):
        input_ids.extend(x[offset:])

    if return_tensors is not None:
        if return_tensors == 'pt':
            return torch.tensor(input_ids, dtype=torch.long)
        raise ValueError(f'Unsupported tensor type: {return_tensors}')
    print("input_ids (return)\n", input_ids)
    return input_ids
"""

In [ ]:
import copy

def preprocess_plain(
    dialog_pair: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    
    print("current file path", "llava/train/train.py")
    print("def preprocess_plain(dialog_pair, tokenizer)")
    print("dialog_pair\n", dialog_pair) # [[{'from': 'human', 'value': '<image>\nGive a brief description of the image.'}, {'from': 'gpt', 'value': 'the divine queen in her elaborate masks canvas print featuring the face and hands of a woman with red hair'}]]
    print("tokenizer\n", type(tokenizer)) # <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>
    # add end signal and concatenate together

    human_dialog = dialog_pair[0]['value']
    print("human_dialog\n", human_dialog) # <image>\nGive a brief description of the image.
    gpt_dialog = dialog_pair[1]['value']
    print("gpt_dialog\n", gpt_dialog) # the divine queen in her elaborate
    default_conversation = conv_templates["plain"] # hard coding
    sep_token = default_conversation.sep # separator_token "\n"

    prompt = DEFAULT_IMAGE_TOKEN + gpt_dialog + sep_token
    print("prompt\n", prompt)
    input_ids = tokenizer_image_token(prompt=prompt, tokenizer=tokenizer, return_tensors='pt')
    target_ids = copy.deepcopy(input_ids)


    tokenized_len = len(tokenizer_image_token(DEFAULT_IMAGE_TOKEN, tokenizer)) # prompt <image>
    print("tokenized_len\n", tokenized_len) # 2
    target_ids[:tokenized_len] = IGNORE_INDEX

    # list処理はその後に重複があるのでコメントアウト
    #input_ids_list = [input_ids]
    #target_ids_list = [target_ids]

    data_dict = dict(input_ids=input_ids, labels=target_ids)

    return data_dict

In [ ]:
"""
def preprocess(
    sources: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
    has_image: bool = False
) -> Dict:

    print("current file path", "llava/train/train.py")
    print("def preprocess(sources, tokenizer, has_image=False)")
    print("sources\n", sources) # [[{'from': 'human', 'value': '<image>\nGive a brief description of the image.'}, {'from': 'gpt', 'value': 'the divine queen in her elaborate masks canvas print featuring the face and hands of a woman with red hair'}]]
    print("tokenizer\n", type(tokenizer)) # <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>
    print("has_image\n", has_image) # True
    
    # Given a list of sources, each is a conversation list. This transform:
    # 1. Add signal '### ' at the beginning each sentence, with end signal '\n';
    # 2. Concatenate conversations together;
    # 3. Tokenize the concatenated conversation;
    # 4. Make a deepcopy as the target. Mask human words with IGNORE_INDEX.
    if default_conversation.sep_style == SeparatorStyle.PLAIN:
        return preprocess_plain(sources, tokenizer) # True
"""

In [ ]:
dialog_pair = [
    {'from': 'human', 'value': '<image>\nRender a clear and concise summary of the photo.'},
    {'from': 'gpt', 'value': 'illustration of a summer holiday in bright colors .'}
]

from transformers import AutoTokenizer
# 例: TinyLlama用
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", use_fast=False)
tokenizer.pad_token = tokenizer.unk_token

data_dict = preprocess_plain(dialog_pair, tokenizer)
data_dict

In [ ]:
import copy
from PIL import Image

# Trainer > def _get_dataloader > dataloader = self.accelerator.prepare(DataLoader(dataset, **dataloader_params))
def __getitem__(self, i) -> Dict[str, torch.Tensor]:

    print("current file path", "llava/train/train.py")
    print("def LazySupervisedDataset.__getitem__(self, i)")
    print("i\n", i) # 0
    data_sample = self.list_data_dict[i]
    dialog_pair = copy.deepcopy(data_sample["conversations"])
    
    # 画像+テキストのサンプル
    if 'image' in data_sample:
        image_file = data_sample['image']
        image_folder = self.data_args.image_folder
        image_processor = self.data_args.image_processor      
        image_path = os.path.join(image_folder, image_file)
        print("image_path\n", image_path)
        try:
            image = Image.open(image_path).convert('RGB')
        except Exception as e:
            print(f"Error opening image: {e}")
            # 画像ファイルのパスがなければこのサンプルは無効としてスキップ
            print("Skipping this sample due to image loading error.")
            return None
        image = image_processor.preprocess(image, return_tensors='pt')['pixel_values'][0]
        dialog_pair = preprocess_multimodal(dialog_pair, self.data_args)
        data_dict = preprocess_plain(dialog_pair, self.tokenizer)
        data_dict['image'] = image
    # テキストのみのサンプル
    elif self.data_args.is_multimodal is False:
        dialog_pair = copy.deepcopy(data_sample["conversations"])
        data_dict = preprocess_plain(dialog_pair, self.tokenizer)
    # image does not exist in the data, but the data_args requires multimodal
    else:
        # 真っ黒の画像をダミー画像として作成する
        crop_size = self.data_args.image_processor.crop_size
        dummy_black_image = torch.zeros(3, crop_size['height'], crop_size['width'])

        dialog_pair = copy.deepcopy(data_sample["conversations"])
        data_dict = preprocess_plain(dialog_pair, self.tokenizer)
        data_dict['image'] = dummy_black_image

    return data_dict

In [ ]:
LazySupervisedDataset.__getitem__ = __getitem__

In [ ]:
"""
import copy

def preprocess_plain(s
    sources: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:

    print("current file path", "llava/train/train.py")
    print("def preprocess_plain(sources, tokenizer)")
    print("sources\n", sources) # [[{'from': 'human', 'value': '<image>\nGive a brief description of the image.'}, {'from': 'gpt', 'value': 'the divine queen in her elaborate masks canvas print featuring the face and hands of a woman with red hair'}]]
    print("tokenizer\n", type(tokenizer)) # <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>
    # add end signal and concatenate together
    conversations = []
    print("conversations initial\n", conversations) # []
    for source in sources:
        print("source current loop\n", source) 
        assert len(source) == 2
        assert DEFAULT_IMAGE_TOKEN in source[0]['value']
        source[0]['value'] = DEFAULT_IMAGE_TOKEN
        conversation = source[0]['value'] + source[1]['value'] + default_conversation.sep
        print("conversation current loop\n", conversation)
        conversations.append(conversation)
    print("conversations (final)\n", conversations) #  ['<image>the divine queen in her elaborate masks canvas print featuring the face and hands of a woman with red hair\n']
    # tokenize conversations
    input_ids = [tokenizer_image_token(prompt, tokenizer, return_tensors='pt') for prompt in conversations]
    print("input_ids\n", input_ids) # [tensor([    1,  -200,   278, 25616, 26624,   297,   902, 19430, 11105, 29879, 10508,  1596, 23425,   278,  3700,   322,  6567,   310,   263,  6114, 411,  2654, 11315,    13])]
    for idx, tensor in enumerate(input_ids):
        if hasattr(tensor, 'shape'):
            print(f"input_ids[{idx}].shape\n", tensor.shape) # torch.Size([24])
    targets = copy.deepcopy(input_ids)
    print("targets\n", targets) # [tensor([    1,  -200,   278, 25616, 26624,   297,   902, 19430, 11105, 29879, 10508,  1596, 23425,   278,  3700,   322,  6567,   310,   263,  6114, 411,  2654, 11315,    13])]
    for idx, tensor in enumerate(targets):
        if hasattr(tensor, 'shape'):
            print(f"targets[{idx}].shape\n", tensor.shape) # torch.Size([24])
    print("sources\n", sources) # [[{'from': 'human', 'value': '<image>'}, {'from': 'gpt', 'value': 'the divine queen in her elaborate masks canvas print featuring the face and hands of a woman with red hair'}]]
    for target, source in zip(targets, sources):
        tokenized_len = len(tokenizer_image_token(source[0]['value'], tokenizer)) # prompt <image>
        target[:tokenized_len] = IGNORE_INDEX

    print("input_ids (return)\n", input_ids) # [tensor([    1,  -200,   278, 25616, 26624,   297,   902, 19430, 11105, 29879, 10508,  1596, 23425,   278,  3700,   322,  6567,   310,   263,  6114, 411,  2654, 11315,    13])]
    print("targets (return)\n", targets) #  [tensor([ -100,  -100,   278, 25616, 26624,   297,   902, 19430, 11105, 29879, 10508,  1596, 23425,   278,  3700,   322,  6567,   310,   263,  6114, 411,  2654, 11315,    13])]
    return dict(input_ids=input_ids, labels=targets)
"""

In [ ]:
sample_dict0 = train_dataset.__getitem__(0)
sample_dict1 = train_dataset.__getitem__(1)
instances = [sample_dict0, sample_dict1]

In [ ]:
"""
import torch

# 擬似データの input_ids, labels
input_ids0 = torch.tensor([1, -200, 4094, 21180, 918, 297, 278, 4272, 13])
labels0 = torch.tensor([-100, -100, 4094, 21180, 918, 297, 278, 4272, 13])
image0 = torch.randn(3, 336, 336)

input_ids1 = torch.tensor([1, -200, 8632, 362, 310, 263, 11801, 8753, 22394, 297, 11785, 11955, 869, 13])
labels1 = torch.tensor([-100, -100, 8632, 362, 310, 263, 11801, 8753, 22394, 297, 11785, 11955, 869, 13])
image1 = torch.randn(3, 336, 336)

instances = [
    {
        "input_ids": input_ids0,
        "labels": labels0,
        "image": image0
    },
    {
        "input_ids": input_ids1 ,
        "labels": labels1,
        "image": image1
    }
]
"""

In [ ]:
instances

In [ ]:
from transformers import AutoTokenizer

# 例: TinyLlama用
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", use_fast=False)
tokenizer.pad_token = tokenizer.unk_token

def custom_decode(input_ids, tokenizer):
    # -200 など特殊トークンは手動で置換
    tokens = []
    for id in input_ids.tolist():
        if id == -200:
            tokens.append("<image>")
        elif id == -100:
            tokens.append("<IGNORE>")
        else:
            tokens.append(tokenizer.decode([id], skip_special_tokens=False))
    return " ".join(tokens)

input_ids0 = instances[0]['input_ids']
input_ids1 = instances[1]['input_ids']
print("input_ids0:", custom_decode(input_ids0, tokenizer))
print("input_ids1:", custom_decode(input_ids1, tokenizer))

In [ ]:
# データローダーが None を返すことがあるので、Noneのサンプルを除外。
instances = [instance for instance in instances if instance is not None]

In [ ]:
input_ids, labels = tuple([instance[key] for instance in instances] 
                          for key in ("input_ids", "labels"))

In [ ]:
print("input_ids\n", input_ids)
print("labels\n", labels)

In [ ]:
# input_idsはtokenizerのpad_token_id(0)でパディング
print("tokenizer.pad_token_id\n", tokenizer.pad_token_id)
input_ids = torch.nn.utils.rnn.pad_sequence(
    input_ids,
    batch_first=True,
    padding_value=tokenizer.pad_token_id
)
print("input_ids\n", input_ids)

In [ ]:
# labelsはIGNORE_INDEX(-100)でパディング
print("IGNORE_INDEX\n", IGNORE_INDEX)
labels = torch.nn.utils.rnn.pad_sequence(
    labels,
    batch_first=True,
    padding_value=IGNORE_INDEX
)
print("labels\n", labels)

In [ ]:
tokenizer.model_max_length

In [ ]:
input_ids = input_ids[:, :tokenizer.model_max_length]
labels = labels[:, :tokenizer.model_max_length]

In [ ]:
print("input_ids\n", input_ids)
print("labels\n", labels)

In [ ]:
attention_mask=input_ids.ne(tokenizer.pad_token_id)
print("attention_mask\n", attention_mask)

In [ ]:
batch = dict(
    input_ids=input_ids,
    labels=labels,
    attention_mask=attention_mask,
)

In [ ]:
batch

In [ ]:
if 'image' in instances[0]:
    # 画像リストを抽出
    images = [instance['image'] for instance in instances]

In [ ]:
images

In [ ]:
all_same_shape = all(
    x is not None and x.shape == images[0].shape for x in images
)

all_same_shape

In [ ]:
# 画像をバッチ化 or リストのまま格納
if all_same_shape:
    batch['images'] = torch.stack(images)
else:
    # リストのまま渡すと困るのではないか？この疑問は未解決
    batch['images'] = images

In [ ]:
batch['images']

In [ ]:
batch

In [ ]:
# 画像のshapeを出力
if isinstance(batch['images'], torch.Tensor):
    print("batch['images'].shape\n", batch['images'].shape)
else:
    print("batch['images'] is a list, first shape:", images[0].shape if images else None)

data_collator とは、複数のサンプルをまとめてデータをバッチ化するもの、という意味です。

In [ ]:
# Trainer > def _get_dataloader > dataloader_params = {..."collate_fn": data_collator,...}
# self.accelerator.prepare(DataLoader(dataset, **dataloader_params)) で呼ばれる

@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:

        print("current file path", "llava/train/train.py")
        print("def DataCollatorForSupervisedDataset.__call__(self, instances)")
        print("instances\n", instances)
        #  [(torch.Size([24]), torch.Size([24]), torch.Size([3, 336, 336]))]
        print("shape of each instance's input_ids and labels, and images(if any):", [(x['input_ids'].shape, x['labels'].shape, x.get('image', None).shape if 'image' in x else None) for x in instances])
        # データローダーが None を返すことがあるので、Noneのサンプルを除外。
        instances = [x for x in instances if x is not None]
        # input_idsとlabelsのそれぞれについてリストを作成。タプルをつくる。
        input_ids, labels = tuple([instance[key] for instance in instances]
                                  for key in ("input_ids", "labels"))
        # input_idsはtokenizerのpad_token_id(0)でパディング
        print("self.tokenizer.pad_token_id\n", self.tokenizer.pad_token_id)
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids,
            batch_first=True,
            padding_value=self.tokenizer.pad_token_id)
        # labelsはIGNORE_INDEX(-100)でパディング
        print("IGNORE_INDEX\n", IGNORE_INDEX)
        labels = torch.nn.utils.rnn.pad_sequence(labels,
                                                 batch_first=True,
                                                 padding_value=IGNORE_INDEX)
        input_ids = input_ids[:, :self.tokenizer.model_max_length]
        print("input_ids.shape (after pad_sequence and truncate)\n", input_ids.shape)
        print("input_ids (after pad_sequence and truncate)\n", input_ids)
        labels = labels[:, :self.tokenizer.model_max_length]
        print("labels.shape (after pad_sequence and truncate)\n", labels.shape)
        print("labels (after pad_sequence and truncate)\n", labels)
        # .ne() は "not equal" → pad_token_id(=0) じゃない部分を 1、pad 部分を 0 にする。モデルが pad 部分を読まないように制御するマスクです。
        attention_mask=input_ids.ne(self.tokenizer.pad_token_id)
        
        batch = dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=attention_mask,
        )

        if 'image' in instances[0]:
            images = [instance['image'] for instance in instances]
            if all(x is not None and x.shape == images[0].shape for x in images):
                batch['images'] = torch.stack(images)
            else:
                batch['images'] = images
            print("batch['images'].shape\n", batch['images'].shape)
        
        print("batch (return)\n", batch)
        print("shape of each batch's input_ids and labels, and images(if any):", [(batch['input_ids'].shape, batch['labels'].shape, batch.get('images', None).shape if 'images' in batch else None)])
        return batch

In [ ]:
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

In [ ]:
data_module = dict(train_dataset=train_dataset, eval_dataset=None, data_collator=data_collator)

In [ ]:
from typing import Dict

def make_supervised_data_module(tokenizer: transformers.PreTrainedTokenizer,
                                data_args) -> Dict:

    print("current file path", "llava/train/train.py")
    print("def make_supervised_data_module(tokenizer, data_args)")
    print("tokenizer\n", type(tokenizer))
    print("data_args\n", data_args) #  DataArguments(data_path='/content/LLaVA/blip_laion_cc_sbu_1.json', lazy_preprocess=True, is_multimodal=True, image_folder='/content/LLaVA/images', image_aspect_ratio='square')
    """Make dataset and collator for supervised fine-tuning."""
    train_dataset = LazySupervisedDataset(tokenizer=tokenizer,
                                data_path=data_args.data_path,
                                data_args=data_args)
    print("train_dataset\n", train_dataset) # <llava.train.train.LazySupervisedDataset object at 0x7ed6341f4880>
    print("len(train_dataset)\n", len(train_dataset)) # 1
    data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
    print("data_collator\n", data_collator) # DataCollatorForSupervisedDataset(tokenizer=LlamaTokenizer(name_or_path='TinyLlama/TinyLlama-1.1B-Chat-v1.0', vocab_size=32000, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False))
    data_module = dict(train_dataset=train_dataset,
                  eval_dataset=None,
                  data_collator=data_collator)
    print("def make_supervised_data_module: result (return)\n", data_module) # {'train_dataset': <llava.train.train.LazySupervisedDataset object at 0x7ed6341f4880>, 'eval_dataset': None, 'data_collator': DataCollatorForSupervisedDataset(tokenizer=LlamaTokenizer(name_or_path='TinyLlama/TinyLlama-1.1B-Chat-v1.0', vocab_size=32000, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False))}
    return data_module

In [ ]:
data_module = make_supervised_data_module(tokenizer=tokenizer, data_args=data_args)
print("data_module\n", data_module)

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_args.model_name_or_path,
    cache_dir=training_args.cache_dir,
    model_max_length=training_args.model_max_length,
    padding_side="right",
    use_fast=False,
)
tokenizer.pad_token = tokenizer.unk_token

print("tokenizer\n", tokenizer)

In [ ]:
train_dataset = LazySupervisedDataset(tokenizer=tokenizer, data_path=data_args.data_path, data_args=data_args)

In [ ]:
sample_dict0 = train_dataset.__getitem__(0)
sample_dict1 = train_dataset.__getitem__(1)
instances = [sample_dict0, sample_dict1]
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
batch = data_collator(instances)
print("batch\n", batch)

In [ ]:
images = batch['images']
print("images shape\n", images.shape) # torch.Size([2, 3, 336, 336])
print("images\n", images)

In [ ]:
@torch.no_grad()
def forward(self, images):
    print("current file path", "llava/llava/model/multimodal_encoder/clip_encoder.py")
    print("def CLIPVisionTower.forward(self, images)")
    print("images shape\n", images.shape) # torch.Size([2, 3, 336, 336])
    images = images.to(device=self.vision_tower.device, dtype=self.vision_tower.dtype)
    vision_outputs = self.vision_tower(images, output_hidden_states=True)
    print("after process vision_outputs\n", type(vision_outputs)) # 24層のtuple
    vision_features = vision_outputs.hidden_states[self.select_layer]
    if self.select_feature == 'patch':
        vision_features = vision_features[:, 1:]
    print("selected vision_feature shape\n", vision_features.shape)
    return vision_features

In [ ]:
CLIPVisionTower.forward = forward

In [ ]:
# LlavaLlamaForCausalLM_model = LlavaLlamaForCausalLM.from_pretrained(model_args.model_name_or_path)
# LlavaLlamaModel_model = self.get_model()
CLIPVisionTower_model = LlavaLlamaModel_model.get_vision_tower()
vision_features = CLIPVisionTower_model.forward(images)
print("vision_features shape\n", vision_features.shape)

In [ ]:
print(vision_features.dtype)
LlavaLlamaModel_model.mm_projector = LlavaLlamaModel_model.mm_projector.to(dtype=vision_features.dtype) # 2段目: Projectorの出力
projected_vision_features = LlavaLlamaModel_model.mm_projector(vision_features)  # 2段目: Projectorの出力
print("projected_vision_features shape\n", projected_vision_features.shape)

In [ ]:
def encode_images(self, images):
    print("current file path", "llava/model/llava_arch.py")
    print("def LlavaBaseForCausalLM(ABC).encode_images(self, images)")
    LlavaLlamaModel_model = self.get_model()
    CLIPVisionTower_model = LlavaLlamaModel_model.get_vision_tower()
    vision_features = CLIPVisionTower_model(images)  # 1段目: Vision Encoderの出力
    print("vision_features shape\n", vision_features.shape)
    # dtypeを揃える
    LlavaLlamaModel_model.mm_projector = LlavaLlamaModel_model.mm_projector.to(dtype=vision_features.dtype)
    projected_vision_features = LlavaLlamaModel_model.mm_projector(vision_features)  # 2段目: Projectorの出力
    print("projected_vision_features (return) shape\n", projected_vision_features.shape)
    del LlavaLlamaModel_model, CLIPVisionTower_model
    return projected_vision_features

In [ ]:
LlavaBaseForCausalLM.encode_images = encode_images

In [ ]:
images = batch['images']
print("images shape\n", images.shape) # torch.Size([2, 3, 336, 336])
projected_vision_features = LlavaLlamaForCausalLM_model.encode_images(images)
print("projected_vision_features shape\n", projected_vision_features.shape)

In [ ]:
import torch

# GPU上で管理されていたので、デバイスも合わせます
device = training_args.device

sample_dict0 = train_dataset.__getitem__(0)
sample_dict1 = train_dataset.__getitem__(1)
instances = [sample_dict0, sample_dict1]

In [ ]:
instances

In [ ]:
sample_dict0 = train_dataset.__getitem__(0)
sample_dict1 = train_dataset.__getitem__(1)
instances = [sample_dict0, sample_dict1]
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
batch = data_collator(instances)
print("batch\n", batch)

In [ ]:
device = training_args.device
input_ids = batch['input_ids'].to(device)
labels = batch['labels'].to(device)
attention_mask = batch['attention_mask'].to(device)
position_ids = None
past_key_values = None
image_sizes = None

In [ ]:
# === 確認 ===
print("input_ids\n", input_ids)
print("labels\n", labels)
print("images shape\n", images.shape)
print("position_ids\n", position_ids)
print("attention_mask\n", attention_mask)
print("past_key_values\n", past_key_values)
print("image_sizes\n", image_sizes)

In [ ]:
projected_vision_features = LlavaLlamaForCausalLM_model.encode_images(images)

In [ ]:
# これが True だと、MM-MLPアダプタ以外のパラメータは凍結（勾配計算しない）されます。
print("training_args.tune_mm_mlp_adapter\n", training_args.tune_mm_mlp_adapter)
# 画像トークンの前後に特別な開始・終了トークン（<im_start>, <im_end> など）を挿入するかどうかのフラグです。
# これが True だと、画像部分を明示的に区切るトークンがプロンプトに追加され、モデルが画像領域をより明確に認識できるようになります。
print("LlavaLlamaModel_model.config.mm_use_im_start_end\n", LlavaLlamaModel_model.config.mm_use_im_start_end)


In [ ]:
# True Falseではなく、1と0で構成されている場合があるので、bool()を用いてTrueとFalseに変換する
attention_mask = attention_mask.bool()

In [ ]:
# remove the padding using attention_mask -- FIXME
print("input_ids before mask\n", input_ids)
print("labels before mask\n", labels)
input_ids = [cur_input_ids[cur_attention_mask] for cur_input_ids, cur_attention_mask in zip(input_ids, attention_mask)]
labels = [cur_labels[cur_attention_mask] for cur_labels, cur_attention_mask in zip(labels, attention_mask)]
print("input_ids after mask\n", input_ids)
print("labels after mask\n", labels)

In [ ]:
new_input_embeds = []
new_labels = []
cur_image_idx = 0

In [ ]:
cur_input_ids = torch.tensor([0, 1, -200])

In [ ]:
torch.where(cur_input_ids == IMAGE_TOKEN_INDEX)

In [ ]:
for batch_idx, cur_input_ids in enumerate(input_ids):
    print("cur_input_ids shape\n", cur_input_ids.shape)   # torch.Size([24])
    print("cur_input_ids\n", cur_input_ids)

    cur_image_token_idx = torch.where(cur_input_ids == IMAGE_TOKEN_INDEX)[0].item()
    print("cur_image_token_idx\n", cur_image_token_idx)  # [1]
    cur_input_ids_len = cur_input_ids.shape[0]
    cur_labels = labels[batch_idx]
    print("cur_labels\n", cur_labels)

    # 画像トークンが1つだけの場合に限定
    # 1回目: 画像トークンの手前まで
    cur_input_ids_noim_0 = cur_input_ids[:cur_image_token_idx]
    cur_labels_noim_0 = cur_labels[:cur_image_token_idx]
    # 2回目: 画像トークンより後
    cur_input_ids_noim_1 = cur_input_ids[cur_image_token_idx+1:]
    cur_labels_noim_1 = cur_labels[cur_image_token_idx+1:]
    cur_input_ids_noim = [cur_input_ids_noim_0, cur_input_ids_noim_1]
    cur_labels_noim = [cur_labels_noim_0, cur_labels_noim_1]

    print("cur_input_ids_noim\n", cur_input_ids_noim)
    print("cur_labels_noim\n", cur_labels_noim)

    split_sizes = [cur_labels_noim_0.shape[0], cur_labels_noim_1.shape[0]]
    print("split_sizes\n", split_sizes)  # [1, 22]
    LlavaLlamaModel_model = LlavaLlamaForCausalLM_model.get_model()
    cur_input_embeds = LlavaLlamaModel_model.embed_tokens(torch.cat(cur_input_ids_noim))
    del LlavaLlamaModel_model
    print("cur_input_embeds shape\n", cur_input_embeds.shape)  # torch.Size([23, 2048])
    print("cur_input_embeds\n", cur_input_embeds)

    cur_input_embeds_no_im = torch.split(cur_input_embeds, split_sizes, dim=0)
    print("cur_input_embeds_no_im\n", cur_input_embeds_no_im)

In [ ]:
"""
for batch_idx, cur_input_ids in enumerate(input_ids):
    print("cur_input_ids shape\n", cur_input_ids.shape)   # torch.Size([24])
    print("cur_input_ids\n", cur_input_ids)

    cur_image_token_idx = torch.where(cur_input_ids == IMAGE_TOKEN_INDEX)[0]
    print("cur_image_token_idx\n", cur_image_token_idx)  # [1]
    cur_input_ids_len = cur_input_ids.shape[0]

    cur_labels = labels[batch_idx]
    print("cur_labels\n", cur_labels)

    # 画像トークンが1つだけの場合に限定
    if len(cur_image_token_idx) == 1:
        idx = cur_image_token_idx.item()
        # 1回目: 画像トークンの手前まで
        cur_input_ids_noim_0 = cur_input_ids[:idx]
        cur_labels_noim_0 = cur_labels[:idx]
        # 2回目: 画像トークンより後
        cur_input_ids_noim_1 = cur_input_ids[idx+1:]
        cur_labels_noim_1 = cur_labels[idx+1:]
        cur_input_ids_noim = [cur_input_ids_noim_0, cur_input_ids_noim_1]
        cur_labels_noim = [cur_labels_noim_0, cur_labels_noim_1]
    else:
        # 画像トークンが0個 or 複数の場合は従来通り
        cur_input_ids_noim = []
        cur_labels_noim = []
        prev_idx = -1
        image_token_indices = [-1] + cur_image_token_idx.tolist() + [cur_input_ids_len]
        for i in range(len(image_token_indices) - 1):
            cur_input_ids_noim.append(cur_input_ids[image_token_indices[i]+1:image_token_indices[i+1]])
            cur_labels_noim.append(cur_labels[image_token_indices[i]+1:image_token_indices[i+1]])

    print("cur_input_ids_noim\n", cur_input_ids_noim)
    print("cur_labels_noim\n", cur_labels_noim)
"""

In [ ]:
def prepare_inputs_labels_for_multimodal(
    self, input_ids, position_ids, attention_mask, past_key_values, labels,
    images, image_sizes=None
):
    print("current file path", "llava/model/llava_arch.py")
    print("def LlavaMetaForCausalLM(ABC).prepare_inputs_labels_for_multimodal(self, input_ids, position_ids, attention_mask, past_key_values, labels, images, image_sizes=None)")  # not found
    print("input_ids\n", input_ids)
    print("position_ids\n", position_ids)  # None
    print("attention_mask\n", attention_mask)
    print("past_key_values\n", past_key_values)  # None
    print("labels\n", labels)
    print("image_sizes\n", image_sizes)  # None

    projected_vision_features = self.encode_images(images)
    print("projected_vision_features after encode_images shape \n", projected_vision_features.shape)  # torch.Size([1, 576, 2048])
        
    attention_mask_original_dtype = attention_mask.dtype
    # bool型に変換する
    attention_mask = attention_mask.bool()
        
    # remove the padding using attention_mask 
    input_ids = [cur_input_ids[cur_attention_mask] for cur_input_ids, cur_attention_mask in zip(input_ids, attention_mask)]
    labels = [cur_labels[cur_attention_mask] for cur_labels, cur_attention_mask in zip(labels, attention_mask)]

    new_input_embeds = []
    new_labels = []
    cur_image_idx = 0

    for batch_idx, cur_input_ids in enumerate(input_ids):
        print("cur_input_ids shape\n", cur_input_ids.shape)   # torch.Size([24])
        print("cur_input_ids\n", cur_input_ids)

        cur_image_token_idx = torch.where(cur_input_ids == IMAGE_TOKEN_INDEX)[0].item()
        print("cur_image_token_idx\n", cur_image_token_idx)  # [1]
        cur_labels = labels[batch_idx]
        print("cur_labels\n", cur_labels)

        # 画像トークンが1つだけの場合に限定
        # 1回目: 画像トークンの手前まで
        cur_input_ids_noim_0 = cur_input_ids[:cur_image_token_idx]
        cur_labels_noim_0 = cur_labels[:cur_image_token_idx]
        # 2回目: 画像トークンより後
        cur_input_ids_noim_1 = cur_input_ids[cur_image_token_idx+1:]
        cur_labels_noim_1 = cur_labels[cur_image_token_idx+1:]
        cur_input_ids_noim = [cur_input_ids_noim_0, cur_input_ids_noim_1]
        cur_labels_noim = [cur_labels_noim_0, cur_labels_noim_1]

        print("cur_input_ids_noim\n", cur_input_ids_noim)
        print("cur_labels_noim\n", cur_labels_noim)
        
        split_sizes = [x.shape[0] for x in cur_labels_noim]
        print("split_sizes\n", split_sizes)  # [1, 22]
        cur_input_embeds = self.get_model().embed_tokens(torch.cat(cur_input_ids_noim))
        print("cur_input_embeds shape\n", cur_input_embeds.shape)  # torch.Size([23, 2048])
        print("cur_input_embeds\n", cur_input_embeds)
        cur_input_embeds_noim = torch.split(cur_input_embeds, split_sizes, dim=0)
        print("cur_input_embeds_no_im\n", cur_input_embeds_noim) # (torch.size[1, 2048], torch.size[22, 2048])


        cur_new_input_embeds = []
        cur_new_labels = []

        cur_new_input_embeds.append(cur_input_embeds_noim[0])
        cur_new_labels.append(cur_labels_noim[0])

        cur_image_features = projected_vision_features[cur_image_idx]
        cur_new_input_embeds.append(cur_image_features)
        # cur_image_features.shape [576, 2048] より、 トークン長さ 576 が IGNORE_INDEX　でマスクされたlabelsを作成する。
        cur_new_labels.append(torch.full((cur_image_features.shape[0],), IGNORE_INDEX, device=cur_labels.device, dtype=cur_labels.dtype))
        cur_image_idx += 1

        cur_new_input_embeds.append(cur_input_embeds_noim[1])
        cur_new_labels.append(cur_labels_noim[1])

        # cur_new_input_embeds
        # [torch.Size([1, 2048]), torch.Size([576, 2048]), torch.Size([12, 2048])]
        # cur_new_labels
        # [torch.Size([1]), torch.Size([576]), torch.Size([12])]
 
        cur_new_input_embeds = [x.to(self.device) for x in cur_new_input_embeds]
        cur_new_input_embeds = torch.cat(cur_new_input_embeds)
        cur_new_labels = [x.to(self.device) for x in cur_new_labels]
        cur_new_labels = torch.cat(cur_new_labels)

        print("cur_new_input_embeds (after cat) shape\n", cur_new_input_embeds.shape)  # torch.Size([589, 2048])
        print("cur_new_labels (after cat) shape\n", cur_new_labels.shape)  # torch.Size([589])

        new_input_embeds.append(cur_new_input_embeds) # [torch.Size([589, 2048])]
        new_labels.append(cur_new_labels) # [torch.Size([589])]
    
    print("new_input_embeds for samples in batch shape\n", [x.shape for x in new_input_embeds])  
    print("new_labels for samples in batch shape\n", [x.shape for x in new_labels])  
    
    # Truncate sequences to max length as image embeddings can make the sequence longer
    tokenizer_model_max_length = getattr(self.config, 'tokenizer_model_max_length', None)
    if tokenizer_model_max_length is not None:
        new_input_embeds = [x[:tokenizer_model_max_length] for x in new_input_embeds]
        new_labels = [x[:tokenizer_model_max_length] for x in new_labels]


    # The length of the longest sample in a batch
    max_len = max(x.shape[0] for x in new_input_embeds)
    batch_size = len(new_input_embeds)
    new_input_embeds_padded = []
    new_labels_padded = torch.full((batch_size, max_len), IGNORE_INDEX, dtype=new_labels[0].dtype, device=new_labels[0].device)
    print("new_labels_padded (before) shape\n", new_labels_padded.shape)
    attention_mask = torch.zeros((batch_size, max_len), dtype=attention_mask.dtype, device=attention_mask.device)
    print("attention_mask (before) shape\n", attention_mask.shape)  
    
    for i, (cur_new_embed, cur_new_labels) in enumerate(zip(new_input_embeds, new_labels)):
        cur_len = cur_new_embed.shape[0]
        # バッチ内で一番長いサンプルの長さ max_len に合わせるため、足りない分だけゼロ埋め（torch.zeros(...)）を右側に追加します。
        # torch.cat((...), dim=0) で、元の埋め込みとゼロ埋めを縦方向（トークン方向）に連結します。
        # こうすることで、全サンプルの埋め込みベクトルが [max_len, hidden_dim] の同じ形になり、torch.stack でバッチ化できます。
        new_input_embeds_padded.append(torch.cat((
            cur_new_embed,
            torch.zeros((max_len - cur_len, cur_new_embed.shape[1]), dtype=cur_new_embed.dtype, device=cur_new_embed.device)
        ), dim=0))

        if cur_len > 0:
            # padding_side は right なので、[0:cur_len] に代入して、右側を残す
            new_labels_padded[i, :cur_len] = cur_new_labels 
            attention_mask[i, :cur_len] = True
        
        print("new_input_embeds_padded (so far) shape\n", [x.shape for x in new_input_embeds_padded])  # [torch.Size([599, 2048])]
        print("new_labels_padded (so far) shape\n", new_labels_padded.shape)  # torch.Size([1, 599])                    
        print("attention_mask (so far) shape\n", attention_mask.shape)  # torch.Size([1, 599])        

    new_input_embeds = torch.stack(new_input_embeds_padded, dim=0)
    print("new_input_embeds (after) shape\n", new_input_embeds.shape)  # torch.Size([1, 599, 2048])
    new_labels = new_labels_padded
    # 元の型に戻す
    attention_mask = attention_mask.to(dtype=attention_mask_original_dtype)
    print("past_key_values (return)\n", past_key_values)  # None
 
    return None, position_ids, attention_mask, past_key_values, new_input_embeds, new_labels

In [ ]:
LlavaBaseForCausalLM.prepare_inputs_labels_for_multimodal = prepare_inputs_labels_for_multimodal

In [ ]:
device = training_args.device
input_ids = batch['input_ids'].to(device)
labels = batch['labels'].to(device)
attention_mask = batch['attention_mask'].to(device)
position_ids = None
past_key_values = None
image_sizes = None

In [ ]:
(
    input_ids,
    position_ids,
    attention_mask,
    past_key_values,
    inputs_embeds,
    labels
) = LlavaLlamaForCausalLM_model.prepare_inputs_labels_for_multimodal(
    input_ids,
    position_ids,
    attention_mask,
    past_key_values,
    labels,
    images,
    image_sizes
)

In [ ]:
def forward(
    self,
    input_ids: torch.LongTensor = None,
    attention_mask: Optional[torch.Tensor] = None,
    position_ids: Optional[torch.LongTensor] = None,
    past_key_values: Optional[List[torch.FloatTensor]] = None,
    inputs_embeds: Optional[torch.FloatTensor] = None,
    labels: Optional[torch.LongTensor] = None,
    use_cache: Optional[bool] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    images: Optional[torch.FloatTensor] = None,
    image_sizes: Optional[List[List[int]]] = None,
    return_dict: Optional[bool] = None,
) -> Union[Tuple, CausalLMOutputWithPast]:

    print("current file path", "llava/llava/model/language_model/llava_llama.py")
    print("def LlavaLlamaForCausalLM.forward(self, input_ids, attention_mask, position_ids, past_key_values, inputs_embeds, labels, use_cache, output_attentions, output_hidden_states, images, image_sizes, return_dict)")
    print("input_ids.shape\n", input_ids.shape)
    print("attention_mask.shape\n", attention_mask.shape)
    print("position_ids\n", position_ids) # None
    print("past_key_values\n", past_key_values) # None
    print("inputs_embeds\n", inputs_embeds) # None
    print("labels\n", labels)
    print("use_cache\n", use_cache) # None
    print("output_attentions\n", output_attentions) # None
    print("output_hidden_states\n", output_hidden_states) # None
    print("images shape\n", images.shape)
    print("image_sizes\n", image_sizes) # None
    print("return_dict\n", return_dict) # None

    print(f"【COND】 inputs_embeds_is_None={inputs_embeds is None}") # True
    if inputs_embeds is None:
        # 【ENTER】
        (
            input_ids,
            position_ids,
            attention_mask,
            past_key_values,
            inputs_embeds,
            labels
        ) = self.prepare_inputs_labels_for_multimodal(
            input_ids,
            position_ids,
            attention_mask,
            past_key_values,
            labels,
            images,
            image_sizes
        )       

    #  LlamaForCausalLM.forward(self, ...)で明示
    # Trainer > def train > def inner_training_loop > def training_step > model(**inputs) > model.forward
    result = LlamaForCausalLM.forward(
        self,
        input_ids=input_ids,
        attention_mask=attention_mask,
        position_ids=position_ids,
        past_key_values=past_key_values,
        inputs_embeds=inputs_embeds,
        labels=labels,
        use_cache=use_cache,
        output_attentions=output_attentions,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict
    )
    print("Return of def LlavaLlamaForCausalLM.forward(self, input_ids, attention_mask, position_ids, past_key_values, inputs_embeds, labels, use_cache, output_attentions, output_hidden_states, images, image_sizes, return_dict)")
    #print("result of LlavaLlamaForCausalLM.forward (return)\n", result)
    print("logits tensor shape  LlavaLlamaForCausalLM.forward\n", result.logits.shape) # torch.Size([1, 599, 32000])
    print("loss (return)  LlavaLlamaForCausalLM.forward \n", result.loss) # tensor(9.2224, grad_fn=<NllLossBackward0>)
    return result

In [ ]:
LlavaLlamaForCausalLM.forward = forward

In [ ]:
device = training_args.device
input_ids = batch['input_ids'].to(device)
labels = batch['labels'].to(device)
attention_mask = batch['attention_mask'].to(device)
position_ids = None
past_key_values = None
image_sizes = None

images = batch['images'].to(device)

inputs = {
    "input_ids": input_ids,
    "attention_mask": attention_mask,
    "labels": labels,
    "images": images,
    "position_ids": None,
    "past_key_values": None,
    "image_sizes": None,
}

In [ ]:
outputs = LlavaLlamaForCausalLM_model(**inputs)

In [ ]:
print(LlavaLlamaForCausalLM_model)

In [ ]:
for param_name, param_tensor in LlavaLlamaForCausalLM_model.named_parameters():
    print(param_name, param_tensor.shape)

In [ ]:
# この関数 maybe_zero_3 は、DeepSpeedのZero Redundancy Optimizer（ZeRO）を使った分散学習時に、
# パラメータ（param）を安全にCPUにコピーして保存できる形に変換するためのものです。


import logging

def maybe_zero_3(param, ignore_status=False, name=None):

    print("current file path", "llava/train/llava_trainer.py")
    print("def maybe_zero_3(param, ignore_status=False, name=None)")
    print("param maybe_zero_3\n", param)
    print("ignore_status maybe_zero_3\n", ignore_status)
    print("name maybe_zero_3\n", name)
    from deepspeed import zero
    from deepspeed.runtime.zero.partition_parameters import ZeroParamStatus
    # パラメータがZeRO管理下である場合
    if hasattr(param, "ds_id"):
        # パラメータが「利用可能でない」状態
        if param.ds_status == ZeroParamStatus.NOT_AVAILABLE:
            # 警告を出す。ただし、パラメータが「利用可能でない」状態を無視する場合は警告しない。
            if not ignore_status:
                logging.warning(f"{name}: param.ds_status != ZeroParamStatus.NOT_AVAILABLE: {param.ds_status}")
        # 分散されているパラメータを集約。
        with zero.GatheredParameters([param]):
            # CPU上にコピー。
            param = param.data.detach().cpu().clone()
            logging.info(f"{name}: param (after GatheredParameters)\n{param}")
    else:
        # パラメータがZeRO管理下ではない場合
        # CPU上にコピー。
        param = param.detach().cpu().clone()
    return param

In [ ]:
def get_mm_adapter_state_maybe_zero_3(named_params, param_keywords):
    print("current file path", "llava/train/llava_trainer.py")
    print("def get_mm_adapter_state_maybe_zero_3(named_params, keys_to_match)")
    print("named_params get_mm_adapter_state_maybe_zero_3\n", named_params)
    print("keys_to_match get_mm_adapter_state_maybe_zero_3\n", param_keywords)
    # 1. 指定したキーに一致するパラメータだけ抽出（初心者向けに分かりやすく）
    matched_params = {}
    for param_key, param_tensor in named_params:
        for param_keyword in param_keywords:
            if param_keyword in param_key:
                matched_params[param_key] = param_tensor
                break  # 一致したら次のkeyへ

    # 2. DeepSpeed分散対応で安全にCPUにコピー（初心者向けにforループで記述）
    adapter_state_dict = {}
    for param_key, param_tensor in matched_params.items():
        param_tensor_to_save = maybe_zero_3(param_tensor, ignore_status=True, name=param_key).cpu()
        adapter_state_dict[param_key] = param_tensor_to_save
    return adapter_state_dict

In [ ]:
param_keywords = ['mm_projector']
model_params = LlavaLlamaForCausalLM_model.named_parameters()
weight_to_save = get_mm_adapter_state_maybe_zero_3(model_params, param_keywords)
print("weight_to_save\n", weight_to_save)

In [ ]:
!rm -rf ./checkpoints

In [ ]:
output_dir = training_args.output_dir
print("output_dir", output_dir)
LlavaLlamaForCausalLM_model.config.save_pretrained(output_dir)
torch.save(weight_to_save, os.path.join(output_dir, f'mm_projector.bin'))

In [ ]:
!ls ./checkpoints
!ls ./checkpoints/Tinyllava-1.1B-pretrain

In [ ]:
from transformers import Trainer

class LLaVATrainer(Trainer):
    # Trainer > _inner_training_loop > _maybe_log_save_evaluate > self._save_checkpoint(model, trial)
    def _save_checkpoint(self, model, trial, metrics=None):

        print("current file path", "llava/train/llava_trainer.py")
        print("def _save_checkpoint(self, model, trial, metrics=None)")
        print("self _save_checkpoint\n", self) # <llava.train.llava_trainer.LLaVATrainer object at 0x7ed6341f4490>
        print("model _save_checkpoint\n", model)
        print("trial _save_checkpoint\n", trial) # None
        print("metrics _save_checkpoint\n", metrics) # None
        print(f"【COND】tune_mm_mlp_adapter={getattr(self.args, 'tune_mm_mlp_adapter', False)}") # True
        if getattr(self.args, 'tune_mm_mlp_adapter', False):
            from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR # chcekpoint
            checkpoint_folder = f"{PREFIX_CHECKPOINT_DIR}-{self.state.global_step}"
            run_dir = self._get_output_dir(trial=trial)
            output_dir = os.path.join(run_dir, checkpoint_folder)

            # Only save Adapter
            param_keywords = ['mm_projector'] # 'vision_resampler'
            weight_to_save = get_mm_adapter_state_maybe_zero_3(self.model.named_parameters(), param_keywords)

            if self.args.local_rank == 0 or self.args.local_rank == -1:
                # モデルconfigファイルを保存
                self.model.config.save_pretrained(output_dir)
                # weight_to_save を mm_projector.binとして保存
                torch.save(weight_to_save, os.path.join(output_dir, f'mm_projector.bin'))

In [ ]:
trainer = LLaVATrainer(model=LlavaLlamaForCausalLM_model, 
                       tokenizer=tokenizer,
                       args=training_args,
                       **data_module)

In [ ]:
!rm -rf ./checkpoints

In [ ]:
trainer._save_checkpoint(model=LlavaLlamaForCausalLM_model, trial=None, metrics=None)

In [ ]:
!ls ./checkpoints
!ls ./checkpoints/Tinyllava-1.1B-pretrain

In [ ]:
def safe_save_model_for_hf_trainer(trainer: transformers.Trainer,
                                   output_dir: str):

    print("current file path", "llava/train/train.py")
    print("def safe_save_model_for_hf_trainer(trainer: transformers.Trainer, output_dir: str)")
    print("trainer safe_save_model_for_hf_trainer\n", type(trainer)) # <class 'llava.train.llava_trainer.LLaVATrainer'>
    print("output_dir safe_save_model_for_hf_trainer\n", output_dir) # ./checkpoints/Tinyllava-1.1B-pretrain
    """Collects the state dict and dump to disk."""

    # A
    # tune_mm_mlp_adapter=True の場合、mm_projector だけを抽出・保存します（Adapterのみ）。
    # mm_projector だけだから、deepspeedかどうかは関係ない
    if getattr(trainer.args, "tune_mm_mlp_adapter", False): # True
        # Only save Adapter
        param_keywords = ['mm_projector']     
        model_params = trainer.model.named_parameters()
        weight_to_save = get_mm_adapter_state_maybe_zero_3(model_params, param_keywords)
        trainer.model.config.save_pretrained(output_dir)

        current_folder = output_dir.split('/')[-1] # Tinyllava-1.1B-pretrain
        parent_folder = os.path.dirname(output_dir) # ./checkpoints
        if trainer.args.local_rank == 0 or trainer.args.local_rank == -1:
            if current_folder.startswith('checkpoint-'): 
                # SKIP
                mm_projector_folder = os.path.join(parent_folder, "mm_projector")
                os.makedirs(mm_projector_folder, exist_ok=True)
                torch.save(weight_to_save, os.path.join(mm_projector_folder, f'{current_folder}.bin'))
            else:
                # ENTER
                torch.save(weight_to_save, os.path.join(output_dir, f'mm_projector.bin'))
        return

    # B
    # tune_mm_mlp_adapter=False の場合、全モデル保存

    # B1
    # DeepSpeedが有効なら torch.cuda.synchronize() → trainer.save_model(output_dir) で全モデル保存。
    # DeepSpeed（分散学習）を使っている場合は、CUDAの同期を取ってから HuggingFace Trainer の標準の save_model を呼びます。
    if trainer.deepspeed:
        torch.cuda.synchronize()
        trainer.save_model(output_dir)
        return

    # B2
    # should_save は HuggingFace Transformers の TrainingArguments で使われる内部フラグです。
    # 分散学習やマルチGPU環境で「このプロセスがモデル保存を担当するかどうか」を判定します。
    # 通常は local_rank == 0（メインプロセス）だけが保存します。
    state_dict = trainer.model.state_dict()
    if trainer.args.should_save:
        cpu_state_dict = {
            key: value.cpu() for key, value in state_dict.items()
        }
        del state_dict
        trainer._save(output_dir, state_dict=cpu_state_dict)  # noqa

In [ ]:
import pathlib

def train():
    print("current file path", "llava/train/train.py")
    print("def train()")
    global local_rank

    parser = transformers.HfArgumentParser(
        (ModelArguments, DataArguments, TrainingArguments)
        )    
    model_args, data_args, training_args = parser.parse_dict(args_dict)
    local_rank = training_args.local_rank
    compute_dtype = (torch.float16 if training_args.fp16 else (torch.bfloat16 if training_args.bf16 else torch.float32))
    print("compute_dtype\n", compute_dtype)

    print(f"【COND】 vision_tower={model_args.vision_tower}")
    # 【ENTER】 vision_tower=openai/clip-vit-large-patch14-336 なので、この分岐に入る
    if model_args.vision_tower is not None:
        # PreTrainedModel.from_pretrained
        model = LlavaLlamaForCausalLM.from_pretrained(
            model_args.model_name_or_path,
            cache_dir=training_args.cache_dir
        )

    # 【ENTER】 gradient_checkpointing=True なので、この分岐に入る
    if training_args.gradient_checkpointing:
        # 【ENTER】 model に enable_input_require_grads メソッドがあるので、この分岐に入る
        if hasattr(model, "enable_input_require_grads"):
            # PreTrainedModel.enable_input_require_grads
            # 元々 全ての重みについて True
            model.enable_input_require_grads()

    tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=training_args.cache_dir,
        model_max_length=training_args.model_max_length,
        padding_side="right",
        use_fast=False,
    )
    
    print("tokenizer defined by AutoTokenizer.from_pretrained \n", tokenizer)
    # 【ENTER】 version=plain なので、この分岐に入る
    tokenizer.pad_token = tokenizer.unk_token

    
    # preprocess_plain で hard coding しているので省略
    #if model_args.version in conv_templates:
        #default_conversation = conv_templates[model_args.version]

    # 【ENTER】 vision_tower=openai/clip-vit-large-patch14-336 なので、この分岐に入る
    if model_args.vision_tower is not None:
        model.get_model().initialize_vision_modules(
            model_args=model_args,
            fsdp=training_args.fsdp
        )

        vision_tower = model.get_vision_tower()
        vision_tower.to(dtype=torch.bfloat16 if training_args.bf16 else torch.float16, device=training_args.device)

        data_args.image_processor = vision_tower.image_processor
        data_args.is_multimodal = True
        model.config.image_aspect_ratio = data_args.image_aspect_ratio
        model.config.tokenizer_padding_side = tokenizer.padding_side
        model.config.tokenizer_model_max_length = tokenizer.model_max_length
        model.config.tune_mm_mlp_adapter = training_args.tune_mm_mlp_adapter = model_args.tune_mm_mlp_adapter

        if model_args.tune_mm_mlp_adapter: # True
            # 【ENTER】 tune_mm_mlp_adapter=True なので、この分岐に入る
            # モデル全体の全パラメータを「学習不可（requires_grad=False）」にする
            # これで通常の重みは全て凍結される
            model.requires_grad_(False)
            for param in model.get_model().mm_projector.parameters():
                # mm_projector（画像特徴量→テキスト特徴量への変換層）の全パラメータだけを「学習可能（requires_grad=True）」に戻す
                # これで mm_projector のみ学習されることになる
                param.requires_grad = True

        model.config.freeze_mm_mlp_adapter = training_args.freeze_mm_mlp_adapter
        model.config.mm_use_im_start_end = data_args.mm_use_im_start_end = model_args.mm_use_im_start_end
        model.config.mm_projector_lr = training_args.mm_projector_lr
        training_args.use_im_start_end = model_args.mm_use_im_start_end
        model.config.mm_use_im_patch_token = model_args.mm_use_im_patch_token
    
    # make_supervised_data_module は LazySupervisedDataset.__getitem__ より先に呼ばれますが、
    # make_supervised_data_module を理解するためには
    data_module = make_supervised_data_module(tokenizer=tokenizer, data_args=data_args)
    print("data_module\n", data_module) # {'train_dataset': <llava.train.train.LazySupervisedDataset object at 0x7ed6341f4880>, 'eval_dataset': None, 'data_collator': DataCollatorForSupervisedDataset(tokenizer=LlamaTokenizer(name_or_path='TinyLlama/TinyLlama-1.1B-Chat-v1.0', vocab_size=32000, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False))}

    trainer = LLaVATrainer(model=model,
                    tokenizer=tokenizer,
                    args=training_args,
                    **data_module)
    
    if list(pathlib.Path(training_args.output_dir).glob("checkpoint-*")):
        trainer.train(resume_from_checkpoint=False)
    else:
        trainer.train()
    trainer.save_state()

    model.config.use_cache = True
    print("model.config.use_cache = True", model.config.use_cache) # True        
    safe_save_model_for_hf_trainer(trainer=trainer, output_dir=training_args.output_dir)

In [ ]:
if __name__ == "__main__":
    train()